![alt text](http://pandas.pydata.org/_static/pandas_logo.png)

<center><h1> PANDAS TABLE OF CONTENTS </h1></center>


## [Time Series / Date functionality](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries) 

- [Overview](#Overview)
- [Time Stamps vs. Time Spans](#Time-Stamps-vs.-Time-Spans)
- [Converting to Timestamps](#Converting-to-Timestamps)
- [Generating Ranges of Timestamps](#Generating-Ranges-of-Timestamps)
- [Timestamp limitations](#Timestamp-limitations)
- [DatetimeIndex](#DatetimeIndex)
- [DateOffset objects](#DateOffset-objects)
- [Time series related instance methods](#Time-series-related-instance-methods)
- [Resampling](#Resampling)
- [Time Span Representation](#Time-Span-Representation)
- [Converting between Representations](#Converting-between-Representations)
- [Representing out of bounds spans](#Representing-out-of-bounds-spans)
- [Time Zone Handling](#Time-Zone-Handling)


pandas has proven very successful as a tool for working with time series data, especially in the financial data analysis space. Using the NumPy datetime64 and timedelta64 dtypes, we have consolidated a large number of features from other Python libraries like scikits.timeseries as well as created a tremendous amount of new functionality for manipulating time series data.

In working with time series data, we will frequently seek to:

* generate sequences of fixed-frequency dates and time spans
* conform or convert time series to a particular frequency
* compute “relative” dates based on various non-standard time increments (e.g. 5 business days before the last business day of the year), or “roll” dates forward or backward

pandas provides a relatively compact and self-contained set of tools for performing the above tasks.

Create a range of dates:

In [346]:
import pandas as pd
import numpy as 
from numpy.random import randn

from datetime import datetime
from datetime import timedelta
from datetime import time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [2]:
# 72 hours starting with midnight Jan 1st, 2011
rng = pd.date_range('1/1/2011', periods = 72, freq = 'H')

In [3]:
rng[:5]

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 01:00:00',
               '2011-01-01 02:00:00', '2011-01-01 03:00:00',
               '2011-01-01 04:00:00'],
              dtype='datetime64[ns]', freq='H')

Index pandas objects with dates:

In [4]:
ts = pd.Series(np.random.rand(len(rng)), index = rng)

In [5]:
ts.head()

2011-01-01 00:00:00    0.147919
2011-01-01 01:00:00    0.910058
2011-01-01 02:00:00    0.000656
2011-01-01 03:00:00    0.559644
2011-01-01 04:00:00    0.615074
Freq: H, dtype: float64

Change frequency and fill gaps:

In [6]:
# to 45 minute frequency 
converted = ts.asfreq('45Min')

In [7]:
converted.head()

2011-01-01 00:00:00    0.147919
2011-01-01 00:45:00         NaN
2011-01-01 01:30:00         NaN
2011-01-01 02:15:00         NaN
2011-01-01 03:00:00    0.559644
Freq: 45T, dtype: float64

In [8]:
# to 45 minute frequency
converted = ts.asfreq('45Min', method = 'pad')

In [9]:
converted.head()

2011-01-01 00:00:00    0.147919
2011-01-01 00:45:00    0.147919
2011-01-01 01:30:00    0.910058
2011-01-01 02:15:00    0.000656
2011-01-01 03:00:00    0.559644
Freq: 45T, dtype: float64

Resample:

In [10]:
# Daily means
ts.resample('D').mean()

2011-01-01    0.555264
2011-01-02    0.540224
2011-01-03    0.329989
Freq: D, dtype: float64

## Overview

Following table shows the type of time-related classes pandas can handle and how to create them.

<table border="1" class="docutils" align="left">>
<colgroup>
<col width="17%" />
<col width="32%" />
<col width="51%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Class</th>
<th class="head">Remarks</th>
<th class="head">How to create</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td><tt class="docutils literal"><span class="pre">Timestamp</span></tt></td>
<td>Represents a single time stamp</td>
<td><tt class="docutils literal"><span class="pre">to_datetime</span></tt>, <tt class="docutils literal"><span class="pre">Timestamp</span></tt></td>
</tr>
<tr class="row-odd"><td><tt class="docutils literal"><span class="pre">DatetimeIndex</span></tt></td>
<td>Index of <tt class="docutils literal"><span class="pre">Timestamp</span></tt></td>
<td><tt class="docutils literal"><span class="pre">to_datetime</span></tt>, <tt class="docutils literal"><span class="pre">date_range</span></tt>, <tt class="docutils literal"><span class="pre">DatetimeIndex</span></tt></td>
</tr>
<tr class="row-even"><td><tt class="docutils literal"><span class="pre">Period</span></tt></td>
<td>Represents a single time span</td>
<td><tt class="docutils literal"><span class="pre">Period</span></tt></td>
</tr>
<tr class="row-odd"><td><tt class="docutils literal"><span class="pre">PeriodIndex</span></tt></td>
<td>Index of <tt class="docutils literal"><span class="pre">Period</span></tt></td>
<td><tt class="docutils literal"><span class="pre">period_range</span></tt>, <tt class="docutils literal"><span class="pre">PeriodIndex</span></tt></td>
</tr>
</tbody>
</table>

## Time Stamps vs. Time Spans

Time-stamped data is the most basic type of timeseries data that associates values with points in time. For pandas objects it means using the points in time.

In [11]:
pd.Timestamp(datetime(2012, 5, 1))

Timestamp('2012-05-01 00:00:00')

In [12]:
pd.Timestamp('2012-05-01')

Timestamp('2012-05-01 00:00:00')

However, in many cases it is more natural to associate things like change variables with a time span instead. The span represented by Period can be specified explicitly, or inferred from datetime string format.

For example:

In [13]:
pd.Period('2011-01')

Period('2011-01', 'M')

In [14]:
pd.Period('2012-05', freq='D')

Period('2012-05-01', 'D')

Timestamp and Period can be the index. Lists of Timestamp and Period are automatically coerce to DatetimeIndex and PeriodIndex respectively.

In [15]:
 dates = [pd.Timestamp('2012-05-01'), pd.Timestamp('2012-05-02'), pd.Timestamp('2012-05-03')]

In [16]:
dates

[Timestamp('2012-05-01 00:00:00'),
 Timestamp('2012-05-02 00:00:00'),
 Timestamp('2012-05-03 00:00:00')]

In [17]:
ts = pd.Series(np.random.randn(3), dates)

In [18]:
ts

2012-05-01    0.209213
2012-05-02   -0.187640
2012-05-03    0.144789
dtype: float64

In [19]:
type(ts.index)

pandas.tseries.index.DatetimeIndex

In [20]:
ts.index

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

In [21]:
periods = [pd.Period('2012-01'), pd.Period('2012-02'), pd.Period('2012-03')]

In [22]:
periods

[Period('2012-01', 'M'), Period('2012-02', 'M'), Period('2012-03', 'M')]

In [23]:
ts = pd.Series(np.random.randn(3), periods)

In [24]:
ts

2012-01    1.011463
2012-02   -1.159356
2012-03   -1.891224
Freq: M, dtype: float64

In [25]:
type(ts.index)

pandas.tseries.period.PeriodIndex

In [26]:
ts.index

PeriodIndex(['2012-01', '2012-02', '2012-03'], dtype='int64', freq='M')

pandas allows you to capture both representations and convert between them. Under the hood, pandas represents timestamps using instances of Timestamp and sequences of timestamps using instances of DatetimeIndex. For regular time spans, pandas uses Period objects for scalar values and PeriodIndex for sequences of spans. Better support for irregular intervals with arbitrary start and end points are forth-coming in future releases.

## Converting to Timestamps

To convert a Series or list-like object of date-like objects e.g. strings, epochs, or a mixture, you can use the to_datetime function. When passed a Series, this returns a Series (with the same index), while a list-like is converted to a DatetimeIndex:

In [27]:
pd.to_datetime(pd.Series(['Jul 31, 2009', '2010-01-10', None]))

0   2009-07-31
1   2010-01-10
2          NaT
dtype: datetime64[ns]

In [28]:
pd.to_datetime(['2005/11/23', '2010.12.31'])

DatetimeIndex(['2005-11-23', '2010-12-31'], dtype='datetime64[ns]', freq=None)

If you use dates which start with the day first (i.e. European style), you can pass the dayfirst flag:

In [29]:
pd.to_datetime(['04-01-2012 10:00'], dayfirst=True)

DatetimeIndex(['2012-01-04 10:00:00'], dtype='datetime64[ns]', freq=None)

In [30]:
pd.to_datetime(['14-01-2012', '01-14-2012'], dayfirst=True)

DatetimeIndex(['2012-01-14', '2012-01-14'], dtype='datetime64[ns]', freq=None)

**Warning:** You see in the above example that dayfirst isn’t strict, so if a date can’t be parsed with the day being first it will be parsed as if dayfirst were False.

**Note:** Specifying a format argument will potentially speed up the conversion considerably and on versions later then 0.13.0 explicitly specifying a format string of ‘%Y%m%d’ takes a faster path still.

If you pass a single string to to_datetime, it returns single Timestamp. Also, Timestamp can accept the string input. Note that Timestamp doesn’t accept string parsing option like dayfirst or format, use to_datetime if these are required.

In [31]:
 pd.to_datetime('2010/11/12')

Timestamp('2010-11-12 00:00:00')

In [32]:
 pd.Timestamp('2010/11/12')

Timestamp('2010-11-12 00:00:00')

New in version 0.18.1.

You can also pass a DataFrame of integer or string columns to assemble into a Series of Timestamps.

In [33]:
df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5],
                   'hour': [2, 3]})

In [34]:
df

,day,hour,month,year
0,4,2,2,2015
1,5,3,3,2016


In [35]:
pd.to_datetime(df)

0   2015-02-04 02:00:00
1   2016-03-05 03:00:00
dtype: datetime64[ns]

You can pass only the columns that you need to assemble.

In [36]:
pd.to_datetime(df[['year', 'month', 'day']])

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

pd.to_datetime looks for standard designations of the datetime component in the column names, including:

* required: year, month, day
* optional: hour, minute, second, millisecond, microsecond, nanosecond

### Invalid Data

**Note:** In version 0.17.0, the default for to_datetime is now errors='raise', rather than errors='ignore'. This means that invalid parsing will raise rather that return the original input as in previous versions.

Pass errors='coerce' to convert invalid data to NaT (not a time):

Raise when unparseable, this is the default

In [37]:
#pd.to_datetime(['2009/07/31', 'asd'], errors='raise')

Return the original input when unparseable

In [38]:
pd.to_datetime(['2009/07/31', 'asd'], errors='ignore')

array(['2009/07/31', 'asd'], dtype=object)

Return NaT for input when unparseable

In [39]:
pd.to_datetime(['2009/07/31', 'asd'], errors='coerce')

DatetimeIndex(['2009-07-31', 'NaT'], dtype='datetime64[ns]', freq=None)

### Epoch Timestamps

It’s also possible to convert integer or float epoch times. The default unit for these is nanoseconds (since these are how Timestamp s are stored). However, often epochs are stored in another unit which can be specified:

Typical epoch stored units

In [40]:
pd.to_datetime([1349720105, 1349806505, 1349892905,
                1349979305, 1350065705], unit='s')

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05',
               '2012-10-12 18:15:05'],
              dtype='datetime64[ns]', freq=None)

In [41]:
pd.to_datetime([1349720105100, 1349720105200, 1349720105300,
                1349720105400, 1349720105500 ], unit='ms')

DatetimeIndex(['2012-10-08 18:15:05.100000', '2012-10-08 18:15:05.200000',
               '2012-10-08 18:15:05.300000', '2012-10-08 18:15:05.400000',
               '2012-10-08 18:15:05.500000'],
              dtype='datetime64[ns]', freq=None)

These work, but the results may be unexpected.

In [42]:
pd.to_datetime([1])

DatetimeIndex(['1970-01-01 00:00:00.000000001'], dtype='datetime64[ns]', freq=None)

In [43]:
pd.to_datetime([1, 3.14], unit='s')

DatetimeIndex(['1970-01-01 00:00:01', '1970-01-01 00:00:03'], dtype='datetime64[ns]', freq=None)

**Note:** Epoch times will be rounded to the nearest nanosecond.

## Generating Ranges of Timestamps

To generate an index with time stamps, you can use either the DatetimeIndex or Index constructor and pass in a list of datetime objects:

In [44]:
dates = [datetime(2012, 5, 1), datetime(2012, 5, 2), datetime(2012, 5, 3)]

In [45]:
dates

[datetime.datetime(2012, 5, 1, 0, 0),
 datetime.datetime(2012, 5, 2, 0, 0),
 datetime.datetime(2012, 5, 3, 0, 0)]

In [46]:
# Note the frequency information
index = pd.DatetimeIndex(dates)

In [47]:
index

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

In [48]:
# Automatically converted to DatetimeIndex
index = pd.Index(dates)

In [49]:
index

DatetimeIndex(['2012-05-01', '2012-05-02', '2012-05-03'], dtype='datetime64[ns]', freq=None)

Practically, this becomes very cumbersome because we often need a very long index with a large number of timestamps. If we need timestamps on a regular frequency, we can use the pandas functions date_range and bdate_range to create timestamp indexes.

In [50]:
index = pd.date_range('2000-1-1', periods = 1000, freq = 'M')

In [51]:
index

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-30',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-30', '2000-10-31',
               ...
               '2082-07-31', '2082-08-31', '2082-09-30', '2082-10-31',
               '2082-11-30', '2082-12-31', '2083-01-31', '2083-02-28',
               '2083-03-31', '2083-04-30'],
              dtype='datetime64[ns]', length=1000, freq='M')

In [52]:
index = pd.bdate_range('2012-1-1', periods = 250)

In [53]:
index

DatetimeIndex(['2012-01-02', '2012-01-03', '2012-01-04', '2012-01-05',
               '2012-01-06', '2012-01-09', '2012-01-10', '2012-01-11',
               '2012-01-12', '2012-01-13',
               ...
               '2012-12-03', '2012-12-04', '2012-12-05', '2012-12-06',
               '2012-12-07', '2012-12-10', '2012-12-11', '2012-12-12',
               '2012-12-13', '2012-12-14'],
              dtype='datetime64[ns]', length=250, freq='B')

Convenience functions like date_range and bdate_range utilize a variety of frequency aliases. The default frequency for date_range is a **calendar day** while the default for bdate_range is a **business day**

In [54]:
start = datetime(2011, 1, 1)
end = datetime(2012, 1, 1)

In [55]:
rng = pd.date_range(start, end)

In [56]:
rng

DatetimeIndex(['2011-01-01', '2011-01-02', '2011-01-03', '2011-01-04',
               '2011-01-05', '2011-01-06', '2011-01-07', '2011-01-08',
               '2011-01-09', '2011-01-10',
               ...
               '2011-12-23', '2011-12-24', '2011-12-25', '2011-12-26',
               '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30',
               '2011-12-31', '2012-01-01'],
              dtype='datetime64[ns]', length=366, freq='D')

In [57]:
rng = pd.bdate_range(start, end)

In [58]:
rng

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14',
               ...
               '2011-12-19', '2011-12-20', '2011-12-21', '2011-12-22',
               '2011-12-23', '2011-12-26', '2011-12-27', '2011-12-28',
               '2011-12-29', '2011-12-30'],
              dtype='datetime64[ns]', length=260, freq='B')

date_range and bdate_range make it easy to generate a range of dates using various combinations of parameters like start, end, periods, and freq:

In [59]:
pd.date_range(start, end, freq='BM')

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BM')

In [60]:
pd.date_range(start, end, freq='W')

DatetimeIndex(['2011-01-02', '2011-01-09', '2011-01-16', '2011-01-23',
               '2011-01-30', '2011-02-06', '2011-02-13', '2011-02-20',
               '2011-02-27', '2011-03-06', '2011-03-13', '2011-03-20',
               '2011-03-27', '2011-04-03', '2011-04-10', '2011-04-17',
               '2011-04-24', '2011-05-01', '2011-05-08', '2011-05-15',
               '2011-05-22', '2011-05-29', '2011-06-05', '2011-06-12',
               '2011-06-19', '2011-06-26', '2011-07-03', '2011-07-10',
               '2011-07-17', '2011-07-24', '2011-07-31', '2011-08-07',
               '2011-08-14', '2011-08-21', '2011-08-28', '2011-09-04',
               '2011-09-11', '2011-09-18', '2011-09-25', '2011-10-02',
               '2011-10-09', '2011-10-16', '2011-10-23', '2011-10-30',
               '2011-11-06', '2011-11-13', '2011-11-20', '2011-11-27',
               '2011-12-04', '2011-12-11', '2011-12-18', '2011-12-25',
               '2012-01-01'],
              dtype='datetime64[ns]', freq='W-S

In [61]:
pd.bdate_range(end=end, periods=20)

DatetimeIndex(['2011-12-05', '2011-12-06', '2011-12-07', '2011-12-08',
               '2011-12-09', '2011-12-12', '2011-12-13', '2011-12-14',
               '2011-12-15', '2011-12-16', '2011-12-19', '2011-12-20',
               '2011-12-21', '2011-12-22', '2011-12-23', '2011-12-26',
               '2011-12-27', '2011-12-28', '2011-12-29', '2011-12-30'],
              dtype='datetime64[ns]', freq='B')

In [62]:
pd.bdate_range(start=start, periods=20)

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07', '2011-01-10', '2011-01-11', '2011-01-12',
               '2011-01-13', '2011-01-14', '2011-01-17', '2011-01-18',
               '2011-01-19', '2011-01-20', '2011-01-21', '2011-01-24',
               '2011-01-25', '2011-01-26', '2011-01-27', '2011-01-28'],
              dtype='datetime64[ns]', freq='B')

The start and end dates are strictly inclusive. So it will not generate any dates outside of those dates if specified.

## Timestamp limitations

Since pandas represents timestamps in nanosecond resolution, the timespan that can be represented using a 64-bit integer is limited to approximately 584 years:

In [63]:
pd.Timestamp.min

Timestamp('1677-09-22 00:12:43.145225')

In [64]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

See [here](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-oob) for ways to represent data outside these bound.

## DatetimeIndex

One of the main uses for DatetimeIndex is as an index for pandas objects. The DatetimeIndex class contains many timeseries related optimizations:

* A large range of dates for various offsets are pre-computed and cached under the hood in order to make generating subsequent date ranges very fast (just have to grab a slice)
* Fast shifting using the shift and tshift method on pandas objects
* Unioning of overlapping DatetimeIndex objects with the same frequency is very fast (important for fast data alignment)
* Quick access to date fields via properties such as year, month, etc.
* Regularization functions like snap and very fast asof logic

DatetimeIndex objects has all the basic functionality of regular Index objects and a smorgasbord of advanced timeseries-specific methods for easy frequency processing.

**See also:** [Reindexing methods](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-reindexing)

**Note:** While pandas does not force you to have a sorted date index, some of these methods may have unexpected or incorrect behavior if the dates are unsorted. So please be careful.

DatetimeIndex can be used like a regular index and offers all of its intelligent functionality like selection, slicing, etc.

In [65]:
rng = pd.date_range(start, end, freq='BM')

In [66]:
rng

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31', '2011-06-30', '2011-07-29', '2011-08-31',
               '2011-09-30', '2011-10-31', '2011-11-30', '2011-12-30'],
              dtype='datetime64[ns]', freq='BM')

In [67]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [68]:
ts

2011-01-31   -0.845581
2011-02-28    0.424287
2011-03-31   -0.849455
2011-04-29   -0.818275
2011-05-31    0.811956
2011-06-30    0.709809
2011-07-29   -1.856310
2011-08-31   -1.783050
2011-09-30   -2.167756
2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

In [69]:
ts[:5].index

DatetimeIndex(['2011-01-31', '2011-02-28', '2011-03-31', '2011-04-29',
               '2011-05-31'],
              dtype='datetime64[ns]', freq='BM')

In [70]:
ts[::2].index

DatetimeIndex(['2011-01-31', '2011-03-31', '2011-05-31', '2011-07-29',
               '2011-09-30', '2011-11-30'],
              dtype='datetime64[ns]', freq='2BM')

### DatetimeIndex Partial String Indexing

You can pass in dates and strings that parse to dates as indexing parameters:

In [71]:
ts['1/31/2011']

-0.84558090870510305

In [72]:
ts[datetime(2011, 10, 1):]

2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

In [73]:
ts['10/31/2011':'12/31/2011']

2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

To provide convenience for accessing longer time series, you can also pass in the year or year and month as strings:

In [74]:
ts['2011']

2011-01-31   -0.845581
2011-02-28    0.424287
2011-03-31   -0.849455
2011-04-29   -0.818275
2011-05-31    0.811956
2011-06-30    0.709809
2011-07-29   -1.856310
2011-08-31   -1.783050
2011-09-30   -2.167756
2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

In [75]:
ts['2011-6']

2011-06-30    0.709809
Freq: BM, dtype: float64

This type of slicing will work on a DataFrame with a DateTimeIndex as well. Since the partial string selection is a form of label slicing, the endpoints **will be** included. This would include matching times on an included date. Here’s an example:

In [76]:
dft = pd.DataFrame(randn(100000,1), #normally np.random.randn
                   columns=['A'],
                   index=pd.date_range('20130101',periods=100000,freq='T'))

In [77]:
dft.tail()

,A
2013-03-11 10:35:00,-0.275275
2013-03-11 10:36:00,0.646475
2013-03-11 10:37:00,0.576389
2013-03-11 10:38:00,-0.331418
2013-03-11 10:39:00,-0.929655


In [78]:
dft['2013'].tail()

,A
2013-03-11 10:35:00,-0.275275
2013-03-11 10:36:00,0.646475
2013-03-11 10:37:00,0.576389
2013-03-11 10:38:00,-0.331418
2013-03-11 10:39:00,-0.929655


This starts on the very first time in the month, and includes the last date & time for the month

In [79]:
dft['2013-1':'2013-2'].tail()

,A
2013-02-28 23:55:00,1.106105
2013-02-28 23:56:00,-0.305561
2013-02-28 23:57:00,-1.693628
2013-02-28 23:58:00,-0.271263
2013-02-28 23:59:00,-0.139506


This specifies a stop time **that includes all of the times on the last day**

In [80]:
dft['2013-1':'2013-2-28'].tail()

,A
2013-02-28 23:55:00,1.106105
2013-02-28 23:56:00,-0.305561
2013-02-28 23:57:00,-1.693628
2013-02-28 23:58:00,-0.271263
2013-02-28 23:59:00,-0.139506


This specifies an **exact** stop time (and is not the same as the above)

In [81]:
dft['2013-1':'2013-2-28 00:00:00'].tail()

,A
2013-02-27 23:56:00,0.335912
2013-02-27 23:57:00,-0.715947
2013-02-27 23:58:00,-0.177943
2013-02-27 23:59:00,-2.050524
2013-02-28 00:00:00,0.034419


We are stopping on the included end-point as it is part of the index

In [82]:
dft['2013-1-15':'2013-1-15 12:30:00'].tail()

,A
2013-01-15 12:26:00,0.969611
2013-01-15 12:27:00,-1.871214
2013-01-15 12:28:00,-0.455386
2013-01-15 12:29:00,1.247758
2013-01-15 12:30:00,-1.934609


New in version 0.18.0.

DatetimeIndex Partial String Indexing also works on DataFrames with a MultiIndex. For example:

In [83]:
dft2 = pd.DataFrame(np.random.randn(20, 1),
                    columns=['A'],
                    index=pd.MultiIndex.from_product([pd.date_range('20130101',
                                                                    periods=10,
                                                                    freq='12H'),
                                                      ['a', 'b']])) 

In [84]:
dft2

A
2013-01-01 00:00:00 a -1.496606
                    b  0.909014
2013-01-01 12:00:00 a -1.710605
                    b  0.687811
2013-01-02 00:00:00 a  0.829332
                    b -0.558779
2013-01-02 12:00:00 a -1.083578
                    b -0.411210
2013-01-03 00:00:00 a -1.365284
                    b  0.979044
2013-01-03 12:00:00 a  1.469849
                    b  0.709814
2013-01-04 00:00:00 a -1.915671
                    b -1.069473
2013-01-04 12:00:00 a -1.095844
                    b  1.012602
2013-01-05 00:00:00 a  1.911478
                    b -0.380957
2013-01-05 12:00:00 a  0.226392
                    b  0.362271

In [85]:
dft2.loc['2013-01-05']

A
2013-01-05 00:00:00 a  1.911478
                    b -0.380957
2013-01-05 12:00:00 a  0.226392
                    b  0.362271

[IndexSlice](http://pandas.pydata.org/pandas-docs/stable/advanced.html#using-slicers)

In [86]:
idx = pd.IndexSlice

In [87]:
idx  

In [88]:
dft2 = dft2.swaplevel(0, 1).sort_index()

In [89]:
dft2

A
a 2013-01-01 00:00:00 -1.496606
  2013-01-01 12:00:00 -1.710605
  2013-01-02 00:00:00  0.829332
  2013-01-02 12:00:00 -1.083578
  2013-01-03 00:00:00 -1.365284
  2013-01-03 12:00:00  1.469849
  2013-01-04 00:00:00 -1.915671
  2013-01-04 12:00:00 -1.095844
  2013-01-05 00:00:00  1.911478
  2013-01-05 12:00:00  0.226392
b 2013-01-01 00:00:00  0.909014
  2013-01-01 12:00:00  0.687811
  2013-01-02 00:00:00 -0.558779
  2013-01-02 12:00:00 -0.411210
  2013-01-03 00:00:00  0.979044
  2013-01-03 12:00:00  0.709814
  2013-01-04 00:00:00 -1.069473
  2013-01-04 12:00:00  1.012602
  2013-01-05 00:00:00 -0.380957
  2013-01-05 12:00:00  0.362271

In [90]:
dft2.loc[idx[:, '2013-01-05'], :]

A
a 2013-01-05 00:00:00  1.911478
  2013-01-05 12:00:00  0.226392
b 2013-01-05 00:00:00 -0.380957
  2013-01-05 12:00:00  0.362271

### Datetime Indexing

Indexing a DateTimeIndex with a partial string depends on the “accuracy” of the period, in other words how specific the interval is in relation to the frequency of the index. In contrast, indexing with datetime objects is exact, because the objects have exact meaning. These also follow the semantics of including both endpoints.

These datetime objects are specific hours, minutes, and seconds even though they were not explicitly specified (they are 0).

In [91]:
dft[datetime(2013, 1, 1):datetime(2013,2,28)].tail()

,A
2013-02-27 23:56:00,0.335912
2013-02-27 23:57:00,-0.715947
2013-02-27 23:58:00,-0.177943
2013-02-27 23:59:00,-2.050524
2013-02-28 00:00:00,0.034419


With no defaults.

In [92]:
dft[datetime(2013, 1, 1, 10, 12, 0):datetime(2013, 2, 28, 10, 12, 0)].tail()

,A
2013-02-28 10:08:00,-0.085784
2013-02-28 10:09:00,-0.800666
2013-02-28 10:10:00,-1.379959
2013-02-28 10:11:00,-1.806622
2013-02-28 10:12:00,-1.306354


### Truncating & Fancy Indexing

A truncate convenience function is provided that is equivalent to slicing:

In [93]:
ts

2011-01-31   -0.845581
2011-02-28    0.424287
2011-03-31   -0.849455
2011-04-29   -0.818275
2011-05-31    0.811956
2011-06-30    0.709809
2011-07-29   -1.856310
2011-08-31   -1.783050
2011-09-30   -2.167756
2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

In [94]:
ts.truncate(before='10/31/2011', after='12/31/2011')

2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

Even complicated fancy indexing that breaks the DatetimeIndex’s frequency regularity will result in a DatetimeIndex (but frequency is lost):

In [95]:
ts[[0, 2, 6]].index

DatetimeIndex(['2011-01-31', '2011-03-31', '2011-07-29'], dtype='datetime64[ns]', freq=None)

### Time/Date Components

There are several time/date properties that one can access from Timestamp or a collection of timestamps like a DateTimeIndex.

<table border="1" class="docutils" align="left">
<colgroup>
<col width="19%" />
<col width="81%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Property</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>year</td>
<td>The year of the datetime</td>
</tr>
<tr class="row-odd"><td>month</td>
<td>The month of the datetime</td>
</tr>
<tr class="row-even"><td>day</td>
<td>The days of the datetime</td>
</tr>
<tr class="row-odd"><td>hour</td>
<td>The hour of the datetime</td>
</tr>
<tr class="row-even"><td>minute</td>
<td>The minutes of the datetime</td>
</tr>
<tr class="row-odd"><td>second</td>
<td>The seconds of the datetime</td>
</tr>
<tr class="row-even"><td>microsecond</td>
<td>The microseconds of the datetime</td>
</tr>
<tr class="row-odd"><td>nanosecond</td>
<td>The nanoseconds of the datetime</td>
</tr>
<tr class="row-even"><td>date</td>
<td>Returns datetime.date</td>
</tr>
<tr class="row-odd"><td>time</td>
<td>Returns datetime.time</td>
</tr>
<tr class="row-even"><td>dayofyear</td>
<td>The ordinal day of year</td>
</tr>
<tr class="row-odd"><td>weekofyear</td>
<td>The week ordinal of the year</td>
</tr>
<tr class="row-even"><td>week</td>
<td>The week ordinal of the year</td>
</tr>
<tr class="row-odd"><td>dayofweek</td>
<td>The numer of the day of the week with Monday=0, Sunday=6</td>
</tr>
<tr class="row-even"><td>weekday</td>
<td>The number of the day of the week with Monday=0, Sunday=6</td>
</tr>
<tr class="row-odd"><td>weekday_name</td>
<td>The name of the day in a week (ex: Friday)</td>
</tr>
<tr class="row-even"><td>quarter</td>
<td>Quarter of the date: Jan=Mar = 1, Apr-Jun = 2, etc.</td>
</tr>
<tr class="row-odd"><td>days_in_month</td>
<td>The number of days in the month of the datetime</td>
</tr>
<tr class="row-even"><td>is_month_start</td>
<td>Logical indicating if first day of month (defined by frequency)</td>
</tr>
<tr class="row-odd"><td>is_month_end</td>
<td>Logical indicating if last day of month (defined by frequency)</td>
</tr>
<tr class="row-even"><td>is_quarter_start</td>
<td>Logical indicating if first day of quarter (defined by frequency)</td>
</tr>
<tr class="row-odd"><td>is_quarter_end</td>
<td>Logical indicating if last day of quarter (defined by frequency)</td>
</tr>
<tr class="row-even"><td>is_year_start</td>
<td>Logical indicating if first day of year (defined by frequency)</td>
</tr>
<tr class="row-odd"><td>is_year_end</td>
<td>Logical indicating if last day of year (defined by frequency)</td>
</tr>
</tbody>
</table>

Furthermore, if you have a Series with datetimelike values, then you can access these properties via the .dt accessor, see the [docs](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dt-accessors)

## DateOffset objects

In the preceding examples, we created DatetimeIndex objects at various frequencies by passing in frequency strings like ‘M’, ‘W’, and ‘BM to the freq keyword. Under the hood, these frequency strings are being translated into an instance of pandas DateOffset, which represents a regular frequency increment. Specific offset logic like “month”, “business day”, or “one hour” is represented in its various subclasses.

<table border="1" class="docutils" align="left">
<colgroup>
<col width="19%" />
<col width="81%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Class name</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>DateOffset</td>
<td>Generic offset class, defaults to 1 calendar day</td>
</tr>
<tr class="row-odd"><td>BDay</td>
<td>business day (weekday)</td>
</tr>
<tr class="row-even"><td>CDay</td>
<td>custom business day (experimental)</td>
</tr>
<tr class="row-odd"><td>Week</td>
<td>one week, optionally anchored on a day of the week</td>
</tr>
<tr class="row-even"><td>WeekOfMonth</td>
<td>the x-th day of the y-th week of each month</td>
</tr>
<tr class="row-odd"><td>LastWeekOfMonth</td>
<td>the x-th day of the last week of each month</td>
</tr>
<tr class="row-even"><td>MonthEnd</td>
<td>calendar month end</td>
</tr>
<tr class="row-odd"><td>MonthBegin</td>
<td>calendar month begin</td>
</tr>
<tr class="row-even"><td>BMonthEnd</td>
<td>business month end</td>
</tr>
<tr class="row-odd"><td>BMonthBegin</td>
<td>business month begin</td>
</tr>
<tr class="row-even"><td>CBMonthEnd</td>
<td>custom business month end</td>
</tr>
<tr class="row-odd"><td>CBMonthBegin</td>
<td>custom business month begin</td>
</tr>
<tr class="row-even"><td>QuarterEnd</td>
<td>calendar quarter end</td>
</tr>
<tr class="row-odd"><td>QuarterBegin</td>
<td>calendar quarter begin</td>
</tr>
<tr class="row-even"><td>BQuarterEnd</td>
<td>business quarter end</td>
</tr>
<tr class="row-odd"><td>BQuarterBegin</td>
<td>business quarter begin</td>
</tr>
<tr class="row-even"><td>FY5253Quarter</td>
<td>retail (aka 52-53 week) quarter</td>
</tr>
<tr class="row-odd"><td>YearEnd</td>
<td>calendar year end</td>
</tr>
<tr class="row-even"><td>YearBegin</td>
<td>calendar year begin</td>
</tr>
<tr class="row-odd"><td>BYearEnd</td>
<td>business year end</td>
</tr>
<tr class="row-even"><td>BYearBegin</td>
<td>business year begin</td>
</tr>
<tr class="row-odd"><td>FY5253</td>
<td>retail (aka 52-53 week) year</td>
</tr>
<tr class="row-even"><td>BusinessHour</td>
<td>business hour</td>
</tr>
<tr class="row-odd"><td>CustomBusinessHour</td>
<td>custom business hour</td>
</tr>
<tr class="row-even"><td>Hour</td>
<td>one hour</td>
</tr>
<tr class="row-odd"><td>Minute</td>
<td>one minute</td>
</tr>
<tr class="row-even"><td>Second</td>
<td>one second</td>
</tr>
<tr class="row-odd"><td>Milli</td>
<td>one millisecond</td>
</tr>
<tr class="row-even"><td>Micro</td>
<td>one microsecond</td>
</tr>
<tr class="row-odd"><td>Nano</td>
<td>one nanosecond</td>
</tr>
</tbody>
</table>

The basic DateOffset takes the same arguments as dateutil.relativedelta, which works like:

In [96]:
d = datetime(2008, 8, 18, 9, 0)

In [97]:
d + relativedelta(months=4, days=5)

datetime.datetime(2008, 12, 23, 9, 0)

We could have done the same thing with DateOffset:

In [98]:
d + DateOffset(months=4, days=5)

Timestamp('2008-12-23 09:00:00')

The key features of a DateOffset object are:

* it can be added / subtracted to/from a datetime object to obtain a shifted date
* it can be multiplied by an integer (positive or negative) so that the increment will be applied multiple times
* it has rollforward and rollback methods for moving a date forward or backward to the next or previous “offset date”

Subclasses of DateOffset define the apply function which dictates custom date increment logic, such as adding business days:

    class BDay(DateOffset):
        """DateOffset increments between business days"""
        def apply(self, other):
            ...

In [99]:
d + BMonthEnd()

Timestamp('2008-08-29 09:00:00')

In [100]:
d - 5 * BDay()

Timestamp('2008-08-11 09:00:00')

The rollforward and rollback methods do exactly what you would expect:

In [101]:
d

datetime.datetime(2008, 8, 18, 9, 0)

In [102]:
offset = BMonthEnd()

In [103]:
offset.rollforward(d)

Timestamp('2008-08-29 09:00:00')

In [104]:
offset.rollback(d)

Timestamp('2008-07-31 09:00:00')

It’s definitely worth exploring the pandas.tseries.offsets module and the various docstrings for the classes.

These operations (apply, rollforward and rollback) preserves time (hour, minute, etc) information by default. To reset time, use normalize=True keyword when creating the offset instance. If normalize=True, result is normalized after the function is applied.

In [105]:
day = Day()

In [106]:
pd.Timestamp('2014-01-01 09:00')

Timestamp('2014-01-01 09:00:00')

In [107]:
day.apply(pd.Timestamp('2014-01-01 09:00'))

Timestamp('2014-01-02 09:00:00')

In [108]:
day = Day(normalize=True)

In [109]:
day.apply(pd.Timestamp('2014-01-01 09:00'))

Timestamp('2014-01-02 00:00:00')

In [110]:
hour = Hour()

In [111]:
hour.apply(pd.Timestamp('2014-01-01 22:00'))

Timestamp('2014-01-01 23:00:00')

In [112]:
hour = Hour(normalize=True)

In [113]:
hour.apply(pd.Timestamp('2014-01-01 22:00'))

Timestamp('2014-01-01 00:00:00')

In [114]:
hour.apply(pd.Timestamp('2014-01-01 23:00'))

Timestamp('2014-01-02 00:00:00')

### Parametric offsets

Some of the offsets can be “parameterized” when created to result in different behaviors. For example, the Week offset for generating weekly data accepts a weekday parameter which results in the generated dates always lying on a particular day of the week:

In [115]:
d

datetime.datetime(2008, 8, 18, 9, 0)

In [116]:
d + Week()

Timestamp('2008-08-25 09:00:00')

In [117]:
d + Week(weekday=4)

Timestamp('2008-08-22 09:00:00')

In [118]:
 (d + Week(weekday=4)).weekday()

4

In [119]:
d - Week()

Timestamp('2008-08-11 09:00:00')

normalize option will be effective for addition and subtraction.

In [120]:
d + Week(normalize=True)

Timestamp('2008-08-25 00:00:00')

In [121]:
d - Week(normalize=True)

Timestamp('2008-08-11 00:00:00')

Another example is parameterizing YearEnd with the specific ending month:

In [122]:
d + YearEnd()

Timestamp('2008-12-31 09:00:00')

In [123]:
d + YearEnd(month=6)

Timestamp('2009-06-30 09:00:00')

### Using offsets with Series / DatetimeIndex

Offsets can be used with either a Series or DatetimeIndex to apply the offset to each element.

In [124]:
rng = pd.date_range('2012-01-01', '2012-01-03')

In [125]:
rng

DatetimeIndex(['2012-01-01', '2012-01-02', '2012-01-03'], dtype='datetime64[ns]', freq='D')

In [126]:
s = pd.Series(rng)

In [127]:
s

0   2012-01-01
1   2012-01-02
2   2012-01-03
dtype: datetime64[ns]

In [128]:
rng + DateOffset(months=2)

DatetimeIndex(['2012-03-01', '2012-03-02', '2012-03-03'], dtype='datetime64[ns]', freq='D')

In [129]:
s + DateOffset(months=2)

0   2012-03-01
1   2012-03-02
2   2012-03-03
dtype: datetime64[ns]

In [130]:
s - DateOffset(months=2)

0   2011-11-01
1   2011-11-02
2   2011-11-03
dtype: datetime64[ns]

If the offset class maps directly to a Timedelta (Day, Hour, Minute, Second, Micro, Milli, Nano) it can be used exactly like a Timedelta - see the [Timedelta section](http://pandas.pydata.org/pandas-docs/stable/timedeltas.html#timedeltas-operations) for more examples.

In [131]:
s - Day(2)

0   2011-12-30
1   2011-12-31
2   2012-01-01
dtype: datetime64[ns]

In [132]:
td = s - pd.Series(pd.date_range('2011-12-29', '2011-12-31'))

In [133]:
td

0   3 days
1   3 days
2   3 days
dtype: timedelta64[ns]

In [134]:
td + Minute(15)

0   3 days 00:15:00
1   3 days 00:15:00
2   3 days 00:15:00
dtype: timedelta64[ns]

Note that some offsets (such as BQuarterEnd) do not have a vectorized implementation. They can still be used but may calculate significantly slower and will raise a PerformanceWarning

In [135]:
rng + BQuarterEnd()

C:\Anaconda2\lib\site-packages\pandas\tseries\index.py:814: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  "or DatetimeIndex", PerformanceWarning)


DatetimeIndex(['2012-03-30', '2012-03-30', '2012-03-30'], dtype='datetime64[ns]', freq=None)

### Custom Business Days (Experimental)

The CDay or CustomBusinessDay class provides a parametric BusinessDay class which can be used to create customized business day calendars which account for local holidays and local weekend conventions.

As an interesting example, let’s look at Egypt where a Friday-Saturday weekend is observed.

In [136]:
from pandas.tseries.offsets import CustomBusinessDay

In [137]:
weekmask_egypt = 'Sun Mon Tue Wed Thu'

In [138]:
# They also observe International Workers' Day so let's
# add that for a couple of years
holidays = ['2012-05-01', datetime(2013, 5, 1), np.datetime64('2014-05-01')]

In [139]:
bday_egypt = CustomBusinessDay(holidays=holidays, weekmask=weekmask_egypt)

In [140]:
dt = datetime(2013, 4, 30)

In [141]:
dt + 2 * bday_egypt

Timestamp('2013-05-05 00:00:00')

Let’s map to the weekday names

In [142]:
dts = pd.date_range(dt, periods=5, freq=bday_egypt)

In [143]:
pd.Series(dts.weekday, dts).map(pd.Series('Mon Tue Wed Thu Fri Sat Sun'.split()))

2013-04-30    Tue
2013-05-02    Thu
2013-05-05    Sun
2013-05-06    Mon
2013-05-07    Tue
Freq: C, dtype: object

As of v0.14 holiday calendars can be used to provide the list of holidays. See the holiday calendar section for more information.

In [144]:
from pandas.tseries.holiday import USFederalHolidayCalendar

In [145]:
 bday_us = CustomBusinessDay(calendar=USFederalHolidayCalendar())

In [146]:
# Friday before MLK Day
dt = datetime(2014, 1, 17)

In [147]:
# Tuesday after MLK Day (Monday is skipped because it's a holiday)
dt + bday_us

Timestamp('2014-01-21 00:00:00')

In [148]:
dt + 2 * bday_us

Timestamp('2014-01-22 00:00:00')

Monthly offsets that respect a certain holiday calendar can be defined in the usual way.

In [149]:
from pandas.tseries.offsets import CustomBusinessMonthBegin

In [150]:
bmth_us = CustomBusinessMonthBegin(calendar=USFederalHolidayCalendar())

In [151]:
# Skip new years
dt = datetime(2013, 12, 17)

In [152]:
dt

datetime.datetime(2013, 12, 17, 0, 0)

In [153]:
dt + bmth_us

Timestamp('2014-01-02 00:00:00')

In [154]:
# Define date index with custom offset
pd.DatetimeIndex(start='20100101',end='20120101',freq=bmth_us)

DatetimeIndex(['2010-01-04', '2010-02-01', '2010-03-01', '2010-04-01',
               '2010-05-03', '2010-06-01', '2010-07-01', '2010-08-02',
               '2010-09-01', '2010-10-01', '2010-11-01', '2010-12-01',
               '2011-01-03', '2011-02-01', '2011-03-01', '2011-04-01',
               '2011-05-02', '2011-06-01', '2011-07-01', '2011-08-01',
               '2011-09-01', '2011-10-03', '2011-11-01', '2011-12-01'],
              dtype='datetime64[ns]', freq='CBMS')

**Note:** The frequency string ‘C’ is used to indicate that a CustomBusinessDay DateOffset is used, it is important to note that since CustomBusinessDay is a parameterised type, instances of CustomBusinessDay may differ and this is not detectable from the ‘C’ frequency string. The user therefore needs to ensure that the ‘C’ frequency string is used consistently within the user’s application.

### Business Hour

The BusinessHour class provides a business hour representation on BusinessDay, allowing to use specific start and end times.

By default, BusinessHour uses 9:00 - 17:00 as business hours. Adding BusinessHour will increment Timestamp by hourly. If target Timestamp is out of business hours, move to the next business hour then increment it. If the result exceeds the business hours end, remaining is added to the next business day.

In [155]:
bh = BusinessHour()

In [156]:
bh

<BusinessHour: BH=09:00-17:00>

In [157]:
# 2014-08-01 is Friday
pd.Timestamp('2014-08-01 10:00').weekday()

4

In [158]:
pd.Timestamp('2014-08-01 10:00') + bh

Timestamp('2014-08-01 11:00:00')

In [159]:
# Below example is the same as: pd.Timestamp('2014-08-01 09:00') + bh
pd.Timestamp('2014-08-01 08:00') + bh

Timestamp('2014-08-01 10:00:00')

In [160]:
# If the results is on the end time, move to the next business day
pd.Timestamp('2014-08-01 16:00') + bh

Timestamp('2014-08-04 09:00:00')

In [161]:
# Remainings are added to the next day
pd.Timestamp('2014-08-01 16:30') + bh

Timestamp('2014-08-04 09:30:00')

In [162]:
# Adding 2 business hours
pd.Timestamp('2014-08-01 10:00') + BusinessHour(2)

Timestamp('2014-08-01 12:00:00')

In [163]:
# Subtracting 3 business hours
pd.Timestamp('2014-08-01 10:00') + BusinessHour(-3)

Timestamp('2014-07-31 15:00:00')

Also, you can specify start and end time by keywords. Argument must be str which has hour:minute representation or datetime.time instance. Specifying seconds, microseconds and nanoseconds as business hour results in ValueError.

In [164]:
bh = BusinessHour(start='11:00', end=time(20, 0))

In [165]:
bh

<BusinessHour: BH=11:00-20:00>

In [166]:
pd.Timestamp('2014-08-01 13:00') + bh

Timestamp('2014-08-01 14:00:00')

In [167]:
pd.Timestamp('2014-08-01 09:00') + bh

Timestamp('2014-08-01 12:00:00')

In [168]:
 pd.Timestamp('2014-08-01 18:00') + bh

Timestamp('2014-08-01 19:00:00')

Passing start time later than end represents midnight business hour. In this case, business hour exceeds midnight and overlap to the next day. Valid business hours are distinguished by whether it started from valid BusinessDay.

In [169]:
bh = BusinessHour(start='17:00', end='09:00')

In [170]:
bh

<BusinessHour: BH=17:00-09:00>

In [171]:
pd.Timestamp('2014-08-01 17:00') + bh

Timestamp('2014-08-01 18:00:00')

In [172]:
pd.Timestamp('2014-08-01 23:00') + bh

Timestamp('2014-08-02 00:00:00')

In [173]:
# Although 2014-08-02 is Satuaday,
# it is valid because it starts from 08-01 (Friday).
pd.Timestamp('2014-08-02 04:00') + bh

Timestamp('2014-08-02 05:00:00')

In [174]:
# Although 2014-08-04 is Monday,
# it is out of business hours because it starts from 08-03 (Sunday)
pd.Timestamp('2014-08-04 04:00') + bh

Timestamp('2014-08-04 18:00:00')

Applying BusinessHour.rollforward and rollback to out of business hours results in the next business hour start or previous day’s end. Different from other offsets, BusinessHour.rollforward may output different results from apply by definition.

This is because one day’s business hour end is equal to next day’s business hour start. For example, under the default business hours (9:00 - 17:00), there is no gap (0 minutes) between 2014-08-01 17:00 and 2014-08-04 09:00.

In [175]:
# This adjusts a Timestamp to business hour edge
BusinessHour().rollback(pd.Timestamp('2014-08-02 15:00'))

Timestamp('2014-08-01 17:00:00')

In [176]:
BusinessHour().rollforward(pd.Timestamp('2014-08-02 15:00'))

Timestamp('2014-08-04 09:00:00')

In [177]:
# It is the same as BusinessHour().apply(pd.Timestamp('2014-08-01 17:00')).
# And it is the same as BusinessHour().apply(pd.Timestamp('2014-08-04 09:00'))
BusinessHour().apply(pd.Timestamp('2014-08-04 15:00'))

Timestamp('2014-08-04 16:00:00')

In [178]:
# BusinessDay results (for reference)
BusinessHour().rollforward(pd.Timestamp('2014-08-02'))

Timestamp('2014-08-04 09:00:00')

In [179]:
# It is the same as BusinessDay().apply(pd.Timestamp('2014-08-01'))
# The result is the same as rollworward because BusinessDay never overlap.
BusinessHour().apply(pd.Timestamp('2014-08-02'))

Timestamp('2014-08-04 10:00:00')

BusinessHour regards Saturday and Sunday as holidays. To use arbitrary holidays, you can use CustomBusinessHour offset, see [Custom Business Hour:](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-custombusinesshour)

### Custom Business Hour

New in version 0.18.1.

The CustomBusinessHour is a mixture of BusinessHour and CustomBusinessDay which allows you to specify arbitrary holidays. CustomBusinessHour works as the same as BusinessHour except that it skips specified custom holidays.

In [180]:
from pandas.tseries.holiday import USFederalHolidayCalendar

In [181]:
bhour_us = CustomBusinessHour(calendar=USFederalHolidayCalendar())

In [182]:
# Friday before MLK Day
dt = datetime(2014, 1, 17, 15)

In [183]:
dt + bhour_us

Timestamp('2014-01-17 16:00:00')

In [184]:
# Tuesday after MLK Day (Monday is skipped because it's a holiday)
dt + bhour_us * 2

Timestamp('2014-01-21 09:00:00')

You can use keyword arguments suported by either BusinessHour and CustomBusinessDay.

In [185]:
bhour_mon = CustomBusinessHour(start='10:00', weekmask='Tue Wed Thu Fri')

In [186]:
# Monday is skipped because it's a holiday, business hour starts from 10:00
dt + bhour_mon * 2

Timestamp('2014-01-21 10:00:00')

### Offset Aliases

A number of string aliases are given to useful common time series frequencies. We will refer to these aliases as offset aliases (referred to as time rules prior to v0.8.0).

<table border="1" class="docutils" align = "left">
<colgroup>
<col width="13%" />
<col width="87%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Alias</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>B</td>
<td>business day frequency</td>
</tr>
<tr class="row-odd"><td>C</td>
<td>custom business day frequency (experimental)</td>
</tr>
<tr class="row-even"><td>D</td>
<td>calendar day frequency</td>
</tr>
<tr class="row-odd"><td>W</td>
<td>weekly frequency</td>
</tr>
<tr class="row-even"><td>M</td>
<td>month end frequency</td>
</tr>
<tr class="row-odd"><td>BM</td>
<td>business month end frequency</td>
</tr>
<tr class="row-even"><td>CBM</td>
<td>custom business month end frequency</td>
</tr>
<tr class="row-odd"><td>MS</td>
<td>month start frequency</td>
</tr>
<tr class="row-even"><td>BMS</td>
<td>business month start frequency</td>
</tr>
<tr class="row-odd"><td>CBMS</td>
<td>custom business month start frequency</td>
</tr>
<tr class="row-even"><td>Q</td>
<td>quarter end frequency</td>
</tr>
<tr class="row-odd"><td>BQ</td>
<td>business quarter endfrequency</td>
</tr>
<tr class="row-even"><td>QS</td>
<td>quarter start frequency</td>
</tr>
<tr class="row-odd"><td>BQS</td>
<td>business quarter start frequency</td>
</tr>
<tr class="row-even"><td>A</td>
<td>year end frequency</td>
</tr>
<tr class="row-odd"><td>BA</td>
<td>business year end frequency</td>
</tr>
<tr class="row-even"><td>AS</td>
<td>year start frequency</td>
</tr>
<tr class="row-odd"><td>BAS</td>
<td>business year start frequency</td>
</tr>
<tr class="row-even"><td>BH</td>
<td>business hour frequency</td>
</tr>
<tr class="row-odd"><td>H</td>
<td>hourly frequency</td>
</tr>
<tr class="row-even"><td>T, min</td>
<td>minutely frequency</td>
</tr>
<tr class="row-odd"><td>S</td>
<td>secondly frequency</td>
</tr>
<tr class="row-even"><td>L, ms</td>
<td>milliseconds</td>
</tr>
<tr class="row-odd"><td>U, us</td>
<td>microseconds</td>
</tr>
<tr class="row-even"><td>N</td>
<td>nanoseconds</td>
</tr>
</tbody>
</table>

### Combining Aliases

As we have seen previously, the alias and the offset instance are fungible in most functions:

In [187]:
pd.date_range(start, periods=5, freq='B')

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07'],
              dtype='datetime64[ns]', freq='B')

In [188]:
pd.date_range(start, periods=5, freq=BDay())

DatetimeIndex(['2011-01-03', '2011-01-04', '2011-01-05', '2011-01-06',
               '2011-01-07'],
              dtype='datetime64[ns]', freq='B')

You can combine together day and intraday offsets:

In [189]:
pd.date_range(start, periods=10, freq='2h20min')

DatetimeIndex(['2011-01-01 00:00:00', '2011-01-01 02:20:00',
               '2011-01-01 04:40:00', '2011-01-01 07:00:00',
               '2011-01-01 09:20:00', '2011-01-01 11:40:00',
               '2011-01-01 14:00:00', '2011-01-01 16:20:00',
               '2011-01-01 18:40:00', '2011-01-01 21:00:00'],
              dtype='datetime64[ns]', freq='140T')

In [190]:
pd.date_range(start, periods=10, freq='1D10U')

DatetimeIndex([       '2011-01-01 00:00:00', '2011-01-02 00:00:00.000010',
               '2011-01-03 00:00:00.000020', '2011-01-04 00:00:00.000030',
               '2011-01-05 00:00:00.000040', '2011-01-06 00:00:00.000050',
               '2011-01-07 00:00:00.000060', '2011-01-08 00:00:00.000070',
               '2011-01-09 00:00:00.000080', '2011-01-10 00:00:00.000090'],
              dtype='datetime64[ns]', freq='86400000010U')

### Anchored Offsets

For some frequencies you can specify an anchoring suffix:

<table border="1" class="docutils" align = "left">
<colgroup>
<col width="13%" />
<col width="87%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Alias</th>
<th class="head">Description</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>W-SUN</td>
<td>weekly frequency (sundays). Same as &#8216;W&#8217;</td>
</tr>
<tr class="row-odd"><td>W-MON</td>
<td>weekly frequency (mondays)</td>
</tr>
<tr class="row-even"><td>W-TUE</td>
<td>weekly frequency (tuesdays)</td>
</tr>
<tr class="row-odd"><td>W-WED</td>
<td>weekly frequency (wednesdays)</td>
</tr>
<tr class="row-even"><td>W-THU</td>
<td>weekly frequency (thursdays)</td>
</tr>
<tr class="row-odd"><td>W-FRI</td>
<td>weekly frequency (fridays)</td>
</tr>
<tr class="row-even"><td>W-SAT</td>
<td>weekly frequency (saturdays)</td>
</tr>
<tr class="row-odd"><td>(B)Q(S)-DEC</td>
<td>quarterly frequency, year ends in December. Same as &#8216;Q&#8217;</td>
</tr>
<tr class="row-even"><td>(B)Q(S)-JAN</td>
<td>quarterly frequency, year ends in January</td>
</tr>
<tr class="row-odd"><td>(B)Q(S)-FEB</td>
<td>quarterly frequency, year ends in February</td>
</tr>
<tr class="row-even"><td>(B)Q(S)-MAR</td>
<td>quarterly frequency, year ends in March</td>
</tr>
<tr class="row-odd"><td>(B)Q(S)-APR</td>
<td>quarterly frequency, year ends in April</td>
</tr>
<tr class="row-even"><td>(B)Q(S)-MAY</td>
<td>quarterly frequency, year ends in May</td>
</tr>
<tr class="row-odd"><td>(B)Q(S)-JUN</td>
<td>quarterly frequency, year ends in June</td>
</tr>
<tr class="row-even"><td>(B)Q(S)-JUL</td>
<td>quarterly frequency, year ends in July</td>
</tr>
<tr class="row-odd"><td>(B)Q(S)-AUG</td>
<td>quarterly frequency, year ends in August</td>
</tr>
<tr class="row-even"><td>(B)Q(S)-SEP</td>
<td>quarterly frequency, year ends in September</td>
</tr>
<tr class="row-odd"><td>(B)Q(S)-OCT</td>
<td>quarterly frequency, year ends in October</td>
</tr>
<tr class="row-even"><td>(B)Q(S)-NOV</td>
<td>quarterly frequency, year ends in November</td>
</tr>
<tr class="row-odd"><td>(B)A(S)-DEC</td>
<td>annual frequency, anchored end of December. Same as &#8216;A&#8217;</td>
</tr>
<tr class="row-even"><td>(B)A(S)-JAN</td>
<td>annual frequency, anchored end of January</td>
</tr>
<tr class="row-odd"><td>(B)A(S)-FEB</td>
<td>annual frequency, anchored end of February</td>
</tr>
<tr class="row-even"><td>(B)A(S)-MAR</td>
<td>annual frequency, anchored end of March</td>
</tr>
<tr class="row-odd"><td>(B)A(S)-APR</td>
<td>annual frequency, anchored end of April</td>
</tr>
<tr class="row-even"><td>(B)A(S)-MAY</td>
<td>annual frequency, anchored end of May</td>
</tr>
<tr class="row-odd"><td>(B)A(S)-JUN</td>
<td>annual frequency, anchored end of June</td>
</tr>
<tr class="row-even"><td>(B)A(S)-JUL</td>
<td>annual frequency, anchored end of July</td>
</tr>
<tr class="row-odd"><td>(B)A(S)-AUG</td>
<td>annual frequency, anchored end of August</td>
</tr>
<tr class="row-even"><td>(B)A(S)-SEP</td>
<td>annual frequency, anchored end of September</td>
</tr>
<tr class="row-odd"><td>(B)A(S)-OCT</td>
<td>annual frequency, anchored end of October</td>
</tr>
<tr class="row-even"><td>(B)A(S)-NOV</td>
<td>annual frequency, anchored end of November</td>
</tr>
</tbody>
</table>

These can be used as arguments to date_range, bdate_range, constructors for DatetimeIndex, as well as various other timeseries-related functions in pandas.

### Anchored Offset Semantics

For those offsets that are anchored to the start or end of specific frequency (MonthEnd, MonthBegin, WeekEnd, etc) the following rules apply to rolling forward and backwards.

When n is not 0, if the given date is not on an anchor point, it snapped to the next(previous) anchor point, and moved |n|-1 additional steps forwards or backwards.

In [191]:
pd.Timestamp('2014-01-02') + MonthBegin(n=1)

Timestamp('2014-02-01 00:00:00')

In [192]:
pd.Timestamp('2014-01-02') + MonthEnd(n=1)

Timestamp('2014-01-31 00:00:00')

In [193]:
pd.Timestamp('2014-01-02') - MonthBegin(n=1)

Timestamp('2014-01-01 00:00:00')

In [194]:
pd.Timestamp('2014-01-02') - MonthEnd(n=1)

Timestamp('2013-12-31 00:00:00')

In [195]:
pd.Timestamp('2014-01-02') + MonthBegin(n=4)

Timestamp('2014-05-01 00:00:00')

In [196]:
pd.Timestamp('2014-01-02') - MonthBegin(n=4)

Timestamp('2013-10-01 00:00:00')

If the given date is on an anchor point, it is moved |n| points forwards or backwards.

In [197]:
pd.Timestamp('2014-01-01') + MonthBegin(n=1)

Timestamp('2014-02-01 00:00:00')

In [198]:
pd.Timestamp('2014-01-31') + MonthEnd(n=1)

Timestamp('2014-02-28 00:00:00')

In [199]:
pd.Timestamp('2014-01-01') - MonthBegin(n=1)

Timestamp('2013-12-01 00:00:00')

In [200]:
pd.Timestamp('2014-01-31') - MonthEnd(n=1)

Timestamp('2013-12-31 00:00:00')

In [201]:
pd.Timestamp('2014-01-01') + MonthBegin(n=4)

Timestamp('2014-05-01 00:00:00')

In [202]:
pd.Timestamp('2014-01-31') - MonthBegin(n=4)

Timestamp('2013-10-01 00:00:00')

For the case when n=0, the date is not moved if on an anchor point, otherwise it is rolled forward to the next anchor point.

In [203]:
pd.Timestamp('2014-01-02') + MonthBegin(n=0)

Timestamp('2014-02-01 00:00:00')

In [204]:
pd.Timestamp('2014-01-02') + MonthEnd(n=0)

Timestamp('2014-01-31 00:00:00')

In [205]:
pd.Timestamp('2014-01-01') + MonthBegin(n=0)

Timestamp('2014-01-01 00:00:00')

In [206]:
pd.Timestamp('2014-01-31') + MonthEnd(n=0)

Timestamp('2014-01-31 00:00:00')

### Legacy Aliases

Note that prior to v0.8.0, time rules had a slightly different look. These are deprecated in v0.17.0, and removed in future version.

<table border="1" class="docutils", align = "left">
<colgroup>
<col width="19%" />
<col width="81%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Legacy Time Rule</th>
<th class="head">Offset Alias</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td>WEEKDAY</td>
<td>B</td>
</tr>
<tr class="row-odd"><td>EOM</td>
<td>BM</td>
</tr>
<tr class="row-even"><td>W&#64;MON</td>
<td>W-MON</td>
</tr>
<tr class="row-odd"><td>W&#64;TUE</td>
<td>W-TUE</td>
</tr>
<tr class="row-even"><td>W&#64;WED</td>
<td>W-WED</td>
</tr>
<tr class="row-odd"><td>W&#64;THU</td>
<td>W-THU</td>
</tr>
<tr class="row-even"><td>W&#64;FRI</td>
<td>W-FRI</td>
</tr>
<tr class="row-odd"><td>W&#64;SAT</td>
<td>W-SAT</td>
</tr>
<tr class="row-even"><td>W&#64;SUN</td>
<td>W-SUN</td>
</tr>
<tr class="row-odd"><td>Q&#64;JAN</td>
<td>BQ-JAN</td>
</tr>
<tr class="row-even"><td>Q&#64;FEB</td>
<td>BQ-FEB</td>
</tr>
<tr class="row-odd"><td>Q&#64;MAR</td>
<td>BQ-MAR</td>
</tr>
<tr class="row-even"><td>A&#64;JAN</td>
<td>BA-JAN</td>
</tr>
<tr class="row-odd"><td>A&#64;FEB</td>
<td>BA-FEB</td>
</tr>
<tr class="row-even"><td>A&#64;MAR</td>
<td>BA-MAR</td>
</tr>
<tr class="row-odd"><td>A&#64;APR</td>
<td>BA-APR</td>
</tr>
<tr class="row-even"><td>A&#64;MAY</td>
<td>BA-MAY</td>
</tr>
<tr class="row-odd"><td>A&#64;JUN</td>
<td>BA-JUN</td>
</tr>
<tr class="row-even"><td>A&#64;JUL</td>
<td>BA-JUL</td>
</tr>
<tr class="row-odd"><td>A&#64;AUG</td>
<td>BA-AUG</td>
</tr>
<tr class="row-even"><td>A&#64;SEP</td>
<td>BA-SEP</td>
</tr>
<tr class="row-odd"><td>A&#64;OCT</td>
<td>BA-OCT</td>
</tr>
<tr class="row-even"><td>A&#64;NOV</td>
<td>BA-NOV</td>
</tr>
<tr class="row-odd"><td>A&#64;DEC</td>
<td>BA-DEC</td>
</tr>
</tbody>
</table>

As you can see, legacy quarterly and annual frequencies are business quarters and business year ends. Please also note the legacy time rule for milliseconds ms versus the new offset alias for month start MS. This means that offset alias parsing is case sensitive.

### Holidays / Holiday Calendars

http://pandas.pydata.org/pandas-docs/stable/timeseries.html#holidays-holiday-calendars

## Time series-related instance methods

### Shifting / lagging

One may want to shift or lag the values in a time series back and forward in time. The method for this is shift, which is available on all of the pandas objects.

In [207]:
ts

2011-01-31   -0.845581
2011-02-28    0.424287
2011-03-31   -0.849455
2011-04-29   -0.818275
2011-05-31    0.811956
2011-06-30    0.709809
2011-07-29   -1.856310
2011-08-31   -1.783050
2011-09-30   -2.167756
2011-10-31   -1.717823
2011-11-30   -0.801152
2011-12-30   -1.959664
Freq: BM, dtype: float64

In [208]:
ts.shift(1)

2011-01-31         NaN
2011-02-28   -0.845581
2011-03-31    0.424287
2011-04-29   -0.849455
2011-05-31   -0.818275
2011-06-30    0.811956
2011-07-29    0.709809
2011-08-31   -1.856310
2011-09-30   -1.783050
2011-10-31   -2.167756
2011-11-30   -1.717823
2011-12-30   -0.801152
Freq: BM, dtype: float64

The shift method accepts an freq argument which can accept a DateOffset class or other timedelta-like object or also a offset alias:

In [209]:
ts.shift(5, freq= 'B') 

2011-02-07   -0.845581
2011-03-07    0.424287
2011-04-07   -0.849455
2011-05-06   -0.818275
2011-06-07    0.811956
2011-07-07    0.709809
2011-08-05   -1.856310
2011-09-07   -1.783050
2011-10-07   -2.167756
2011-11-07   -1.717823
2011-12-07   -0.801152
2012-01-06   -1.959664
dtype: float64

In [210]:
#Optional
ts.shift(5, freq= BDay()) 

2011-02-07   -0.845581
2011-03-07    0.424287
2011-04-07   -0.849455
2011-05-06   -0.818275
2011-06-07    0.811956
2011-07-07    0.709809
2011-08-05   -1.856310
2011-09-07   -1.783050
2011-10-07   -2.167756
2011-11-07   -1.717823
2011-12-07   -0.801152
2012-01-06   -1.959664
dtype: float64

In [211]:
#Optional
ts.shift(5, freq= pd.datetools.bday)

2011-02-07   -0.845581
2011-03-07    0.424287
2011-04-07   -0.849455
2011-05-06   -0.818275
2011-06-07    0.811956
2011-07-07    0.709809
2011-08-05   -1.856310
2011-09-07   -1.783050
2011-10-07   -2.167756
2011-11-07   -1.717823
2011-12-07   -0.801152
2012-01-06   -1.959664
dtype: float64

In [212]:
ts.shift(5, freq='BM')

2011-06-30   -0.845581
2011-07-29    0.424287
2011-08-31   -0.849455
2011-09-30   -0.818275
2011-10-31    0.811956
2011-11-30    0.709809
2011-12-30   -1.856310
2012-01-31   -1.783050
2012-02-29   -2.167756
2012-03-30   -1.717823
2012-04-30   -0.801152
2012-05-31   -1.959664
Freq: BM, dtype: float64

Rather than changing the alignment of the data and the index, DataFrame and Series objects also have a tshift convenience method that changes all the dates in the index by a specified number of offsets:

In [213]:
ts.tshift(5, freq='D')

2011-02-05   -0.845581
2011-03-05    0.424287
2011-04-05   -0.849455
2011-05-04   -0.818275
2011-06-05    0.811956
2011-07-05    0.709809
2011-08-03   -1.856310
2011-09-05   -1.783050
2011-10-05   -2.167756
2011-11-05   -1.717823
2011-12-05   -0.801152
2012-01-04   -1.959664
dtype: float64

Note that with tshift, the leading entry is no longer NaN because the data is not being realigned.

### Frequency conversion

The primary function for changing frequencies is the asfreq function. For a DatetimeIndex, this is basically just a thin, but convenient wrapper around reindex which generates a date_range and calls reindex.

In [214]:
dr = pd.date_range('1/1/2010', periods=3, freq=3 * pd.datetools.bday)

In [215]:
dr

DatetimeIndex(['2010-01-01', '2010-01-06', '2010-01-11'], dtype='datetime64[ns]', freq='3B')

In [216]:
ts = pd.Series(randn(3), index=dr)

In [217]:
ts

2010-01-01    0.792689
2010-01-06   -2.354597
2010-01-11   -0.553942
Freq: 3B, dtype: float64

In [218]:
ts.asfreq(BDay())

2010-01-01    0.792689
2010-01-04         NaN
2010-01-05         NaN
2010-01-06   -2.354597
2010-01-07         NaN
2010-01-08         NaN
2010-01-11   -0.553942
Freq: B, dtype: float64

asfreq provides a further convenience so you can specify an interpolation method for any gaps that may appear after the frequency conversion

In [219]:
ts.asfreq(BDay(), method='pad')

2010-01-01    0.792689
2010-01-04    0.792689
2010-01-05    0.792689
2010-01-06   -2.354597
2010-01-07   -2.354597
2010-01-08   -2.354597
2010-01-11   -0.553942
Freq: B, dtype: float64

### Filling forward / backward

Related to asfreq and reindex is the fillna function documented in the [missing data section](http://pandas.pydata.org/pandas-docs/stable/missing_data.html#missing-data-fillna).

### Converting to Python datetimes

`DatetimeIndex` can be converted to an array of Python native datetime.datetime objects using the `to_pydatetime` method.

## Resampling

**Warning:** The interface to .resample has changed in 0.18.0 to be more groupby-like and hence more flexible. See the whatsnew docs for a comparison with prior versions.

Pandas has a simple, powerful, and efficient functionality for performing resampling operations during frequency conversion (e.g., converting secondly data into 5-minutely data). This is extremely common in, but not limited to, financial applications.

`resample` is a time-based groupby, followed by a reduction method on each of its groups.

See some [cookbook examples](http://pandas.pydata.org/pandas-docs/stable/cookbook.html#cookbook-resample) for some advanced strategies

In [220]:
rng = pd.date_range('1/1/2012', periods=100, freq='S')

In [221]:
ts = pd.Series(np.random.randint(0, 500, len(rng)), index=rng)

In [222]:
ts

2012-01-01 00:00:00    186
2012-01-01 00:00:01    396
2012-01-01 00:00:02     19
2012-01-01 00:00:03    362
2012-01-01 00:00:04    306
2012-01-01 00:00:05    242
2012-01-01 00:00:06    131
2012-01-01 00:00:07    428
2012-01-01 00:00:08    181
2012-01-01 00:00:09    190
2012-01-01 00:00:10    205
2012-01-01 00:00:11    203
2012-01-01 00:00:12    350
2012-01-01 00:00:13     38
2012-01-01 00:00:14     10
2012-01-01 00:00:15    178
2012-01-01 00:00:16    255
2012-01-01 00:00:17    358
2012-01-01 00:00:18    173
2012-01-01 00:00:19    493
2012-01-01 00:00:20    452
2012-01-01 00:00:21     85
2012-01-01 00:00:22    342
2012-01-01 00:00:23    165
2012-01-01 00:00:24    471
2012-01-01 00:00:25    205
2012-01-01 00:00:26    265
2012-01-01 00:00:27    460
2012-01-01 00:00:28    436
2012-01-01 00:00:29    278
                      ... 
2012-01-01 00:01:10    422
2012-01-01 00:01:11    274
2012-01-01 00:01:12    493
2012-01-01 00:01:13     11
2012-01-01 00:01:14    458
2012-01-01 00:01:15    266
2

In [223]:
ts.resample('5Min').sum()

2012-01-01    26979
Freq: 5T, dtype: int32

The resample function is very flexible and allows you to specify many different parameters to control the frequency conversion and resampling operation.

The how parameter can be a function name or numpy array function that takes an array and produces aggregated values:

In [224]:
ts.resample('5Min').mean()

2012-01-01    269.79
Freq: 5T, dtype: float64

In [225]:
ts.resample('5Min').ohlc()

,open,high,low,close
2012-01-01,186,493,5,421


In [226]:
 ts.resample('5Min').max()

2012-01-01    493
Freq: 5T, dtype: int32

Any function available via dispatching can be given to the how parameter by name, including sum, mean, std, sem, max, min, median, first, last, ohlc.

For downsampling, closed can be set to ‘left’ or ‘right’ to specify which end of the interval is closed:

In [227]:
ts.resample('5Min', closed='right').mean()

2011-12-31 23:55:00    186.000000
2012-01-01 00:00:00    270.636364
Freq: 5T, dtype: float64

In [228]:
ts.resample('5Min', closed='left').mean()

2012-01-01    269.79
Freq: 5T, dtype: float64

Parameters like label and loffset are used to manipulate the resulting labels. label specifies whether the result is labeled with the beginning or the end of the interval. loffset performs a time adjustment on the output labels.

In [229]:
ts.resample('5Min').mean() # by default label='right'

2012-01-01    269.79
Freq: 5T, dtype: float64

In [230]:
ts.resample('5Min', label='left').mean()

2012-01-01    269.79
Freq: 5T, dtype: float64

In [231]:
ts.resample('5Min', label='left', loffset='1s').mean()

2012-01-01 00:00:01    269.79
dtype: float64

The axis parameter can be set to 0 or 1 and allows you to resample the specified axis for a DataFrame.

kind can be set to ‘timestamp’ or ‘period’ to convert the resulting index to/from time-stamp and time-span representations. By default resample retains the input representation.

convention can be set to ‘start’ or ‘end’ when resampling period data (detail below). It specifies how low frequency periods are converted to higher frequency periods.

### Up Sampling

For upsampling, you can specify a way to upsample and the limit parameter to interpolate over the gaps that are created:

In [232]:
# from secondly to every 250 milliseconds
ts[:2].resample('250L').asfreq()

2012-01-01 00:00:00.000    186.0
2012-01-01 00:00:00.250      NaN
2012-01-01 00:00:00.500      NaN
2012-01-01 00:00:00.750      NaN
2012-01-01 00:00:01.000    396.0
Freq: 250L, dtype: float64

In [233]:
ts[:2].resample('250L').ffill()

2012-01-01 00:00:00.000    186
2012-01-01 00:00:00.250    186
2012-01-01 00:00:00.500    186
2012-01-01 00:00:00.750    186
2012-01-01 00:00:01.000    396
Freq: 250L, dtype: int32

In [234]:
ts[:2].resample('250L').ffill(limit=2)

2012-01-01 00:00:00.000    186.0
2012-01-01 00:00:00.250    186.0
2012-01-01 00:00:00.500    186.0
2012-01-01 00:00:00.750      NaN
2012-01-01 00:00:01.000    396.0
Freq: 250L, dtype: float64

### Sparse Sampling

Sparse timeseries are ones where you have a lot fewer points relative to the amount of time you are looking to resample. Naively upsampling a sparse series can potentially generate lots of intermediate values. When you don’t want to use a method to fill these values, e.g. fill_method is None, then intermediate values will be filled with NaN.

Since resample is a time-based groupby, the following is a method to efficiently resample only the groups that are not all NaN

In [235]:
rng = pd.date_range('2014-1-1', periods=100, freq='D') + pd.Timedelta('1s')

In [236]:
ts = pd.Series(range(100), index=rng)

If we want to resample to the full range of the series

In [237]:
ts.resample('1M').sum()

2014-01-31     465
2014-02-28    1246
2014-03-31    2294
2014-04-30     945
Freq: M, dtype: int64

In [238]:
ts.resample('3T').sum()

2014-01-01 00:00:00     0.0
2014-01-01 00:03:00     NaN
2014-01-01 00:06:00     NaN
2014-01-01 00:09:00     NaN
2014-01-01 00:12:00     NaN
2014-01-01 00:15:00     NaN
2014-01-01 00:18:00     NaN
2014-01-01 00:21:00     NaN
2014-01-01 00:24:00     NaN
2014-01-01 00:27:00     NaN
2014-01-01 00:30:00     NaN
2014-01-01 00:33:00     NaN
2014-01-01 00:36:00     NaN
2014-01-01 00:39:00     NaN
2014-01-01 00:42:00     NaN
2014-01-01 00:45:00     NaN
2014-01-01 00:48:00     NaN
2014-01-01 00:51:00     NaN
2014-01-01 00:54:00     NaN
2014-01-01 00:57:00     NaN
2014-01-01 01:00:00     NaN
2014-01-01 01:03:00     NaN
2014-01-01 01:06:00     NaN
2014-01-01 01:09:00     NaN
2014-01-01 01:12:00     NaN
2014-01-01 01:15:00     NaN
2014-01-01 01:18:00     NaN
2014-01-01 01:21:00     NaN
2014-01-01 01:24:00     NaN
2014-01-01 01:27:00     NaN
                       ... 
2014-04-09 22:33:00     NaN
2014-04-09 22:36:00     NaN
2014-04-09 22:39:00     NaN
2014-04-09 22:42:00     NaN
2014-04-09 22:45:00 

We can instead only resample those groups where we have points as follows:

In [239]:
from functools import partial
from pandas.tseries.frequencies import to_offset

In [240]:
def round(t, freq):
    freq = to_offset(freq)
    return pd.Timestamp((t.value // freq.delta.value) * freq.delta.value)

In [241]:
ts.groupby(partial(round, freq='3T')).sum()

2014-01-01     0
2014-01-02     1
2014-01-03     2
2014-01-04     3
2014-01-05     4
2014-01-06     5
2014-01-07     6
2014-01-08     7
2014-01-09     8
2014-01-10     9
2014-01-11    10
2014-01-12    11
2014-01-13    12
2014-01-14    13
2014-01-15    14
2014-01-16    15
2014-01-17    16
2014-01-18    17
2014-01-19    18
2014-01-20    19
2014-01-21    20
2014-01-22    21
2014-01-23    22
2014-01-24    23
2014-01-25    24
2014-01-26    25
2014-01-27    26
2014-01-28    27
2014-01-29    28
2014-01-30    29
              ..
2014-03-12    70
2014-03-13    71
2014-03-14    72
2014-03-15    73
2014-03-16    74
2014-03-17    75
2014-03-18    76
2014-03-19    77
2014-03-20    78
2014-03-21    79
2014-03-22    80
2014-03-23    81
2014-03-24    82
2014-03-25    83
2014-03-26    84
2014-03-27    85
2014-03-28    86
2014-03-29    87
2014-03-30    88
2014-03-31    89
2014-04-01    90
2014-04-02    91
2014-04-03    92
2014-04-04    93
2014-04-05    94
2014-04-06    95
2014-04-07    96
2014-04-08    

### Aggregation

Similar to [groupby aggregates](http://pandas.pydata.org/pandas-docs/stable/groupby.html#groupby-aggregate) and the [window functions](http://pandas.pydata.org/pandas-docs/stable/computation.html#stats-aggregate), a Resampler can be selectively resampled.

Resampling a DataFrame, the default will be to act on all columns with the same function.

In [242]:
df = pd.DataFrame(np.random.randn(1000, 3),
                   index=pd.date_range('1/1/2012', freq='S', periods=1000),
                   columns=['A', 'B', 'C'])

In [243]:
df.head(10)

,A,B,C
2012-01-01 00:00:00,-0.390229,0.785164,0.095812
2012-01-01 00:00:01,0.124035,0.708183,-0.843850
2012-01-01 00:00:02,-0.084347,0.744211,-0.617065
2012-01-01 00:00:03,-0.676501,-0.533311,1.305320
2012-01-01 00:00:04,-0.325289,1.109043,0.638275
2012-01-01 00:00:05,0.581369,-0.935268,-0.473533
2012-01-01 00:00:06,-0.639023,0.016967,0.620778
2012-01-01 00:00:07,-1.737969,1.778922,-2.212012
2012-01-01 00:00:08,1.289322,-0.134208,-0.811150
2012-01-01 00:00:09,-0.101274,1.218381,-1.454157


In [244]:
r = df.resample('3T')

In [245]:
r.mean()

,A,B,C
2012-01-01 00:00:00,-0.063918,-0.032462,-0.067516
2012-01-01 00:03:00,-0.023269,0.021582,-0.134601
2012-01-01 00:06:00,-0.015967,0.098343,-0.034806
2012-01-01 00:09:00,-0.037702,0.040599,-0.117842
2012-01-01 00:12:00,-0.002097,0.056075,0.000702
2012-01-01 00:15:00,0.191086,-0.001695,0.026347


We can select a specific column or columns using standard getitem

In [246]:
r['A'].mean()

2012-01-01 00:00:00   -0.063918
2012-01-01 00:03:00   -0.023269
2012-01-01 00:06:00   -0.015967
2012-01-01 00:09:00   -0.037702
2012-01-01 00:12:00   -0.002097
2012-01-01 00:15:00    0.191086
Freq: 3T, Name: A, dtype: float64

In [247]:
r[['A','B']].mean()

,A,B
2012-01-01 00:00:00,-0.063918,-0.032462
2012-01-01 00:03:00,-0.023269,0.021582
2012-01-01 00:06:00,-0.015967,0.098343
2012-01-01 00:09:00,-0.037702,0.040599
2012-01-01 00:12:00,-0.002097,0.056075
2012-01-01 00:15:00,0.191086,-0.001695


You can pass a list or dict of functions to do aggregation with, outputting a DataFrame:

In [248]:
r['A'].agg([np.sum, np.mean, np.std])

,sum,mean,std
2012-01-01 00:00:00,-11.505209,-0.063918,1.039035
2012-01-01 00:03:00,-4.188494,-0.023269,1.019935
2012-01-01 00:06:00,-2.873976,-0.015967,1.038111
2012-01-01 00:09:00,-6.786396,-0.037702,0.948750
2012-01-01 00:12:00,-0.377534,-0.002097,1.003498
2012-01-01 00:15:00,19.108601,0.191086,0.978669


If a dict is passed, the keys will be used to name the columns. Otherwise the function’s name (stored in the function object) will be used.

In [249]:
r['A'].agg({'result1' : np.sum,
            'result2' : np.mean})

,result2,result1
2012-01-01 00:00:00,-0.063918,-11.505209
2012-01-01 00:03:00,-0.023269,-4.188494
2012-01-01 00:06:00,-0.015967,-2.873976
2012-01-01 00:09:00,-0.037702,-6.786396
2012-01-01 00:12:00,-0.002097,-0.377534
2012-01-01 00:15:00,0.191086,19.108601


In [250]:
r.agg([np.sum, np.mean])

A                    B                    C  \
                           sum      mean        sum      mean        sum   
2012-01-01 00:00:00 -11.505209 -0.063918  -5.843169 -0.032462 -12.152835   
2012-01-01 00:03:00  -4.188494 -0.023269   3.884712  0.021582 -24.228142   
2012-01-01 00:06:00  -2.873976 -0.015967  17.701660  0.098343  -6.265138   
2012-01-01 00:09:00  -6.786396 -0.037702   7.307878  0.040599 -21.211606   
2012-01-01 00:12:00  -0.377534 -0.002097  10.093585  0.056075   0.126399   
2012-01-01 00:15:00  19.108601  0.191086  -0.169546 -0.001695   2.634725   

                               
                         mean  
2012-01-01 00:00:00 -0.067516  
2012-01-01 00:03:00 -0.134601  
2012-01-01 00:06:00 -0.034806  
2012-01-01 00:09:00 -0.117842  
2012-01-01 00:12:00  0.000702  
2012-01-01 00:15:00  0.026347

By passing a dict to aggregate you can apply a different aggregation to the columns of a DataFrame:

In [251]:
r.agg({'A' : np.sum,
       'B' : lambda x: np.std(x, ddof=1)})

,A,B
2012-01-01 00:00:00,-11.505209,1.030873
2012-01-01 00:03:00,-4.188494,0.942422
2012-01-01 00:06:00,-2.873976,0.989605
2012-01-01 00:09:00,-6.786396,1.030471
2012-01-01 00:12:00,-0.377534,1.041621
2012-01-01 00:15:00,19.108601,0.911995


The function names can also be strings. In order for a string to be valid it must be implemented on the Resampled object

In [252]:
r.agg({'A' : 'sum', 'B' : 'std'})

,A,B
2012-01-01 00:00:00,-11.505209,1.030873
2012-01-01 00:03:00,-4.188494,0.942422
2012-01-01 00:06:00,-2.873976,0.989605
2012-01-01 00:09:00,-6.786396,1.030471
2012-01-01 00:12:00,-0.377534,1.041621
2012-01-01 00:15:00,19.108601,0.911995


Furthermore, you can also specify multiple aggregation functions for each column separately.

In [253]:
r.agg({'A' : ['sum','std'], 'B' : ['mean','std'] })

A                   B          
                           sum       std      mean       std
2012-01-01 00:00:00 -11.505209  1.039035 -0.032462  1.030873
2012-01-01 00:03:00  -4.188494  1.019935  0.021582  0.942422
2012-01-01 00:06:00  -2.873976  1.038111  0.098343  0.989605
2012-01-01 00:09:00  -6.786396  0.948750  0.040599  1.030471
2012-01-01 00:12:00  -0.377534  1.003498  0.056075  1.041621
2012-01-01 00:15:00  19.108601  0.978669 -0.001695  0.911995

## Time Span Representation

Regular intervals of time are represented by Period objects in pandas while sequences of Period objects are collected in a PeriodIndex, which can be created with the convenience function period_range.

### Period

A Period represents a span of time (e.g., a day, a month, a quarter, etc). You can specify the span via freq keyword using a frequency alias like below. Because freq represents a span of Period, it cannot be negative like “-3D”.

In [254]:
pd.Period('2012', freq='A-DEC')

Period('2012', 'A-DEC')

In [255]:
pd.Period('2012-1-1', freq='D')

Period('2012-01-01', 'D')

In [256]:
pd.Period('2012-1-1 19:00', freq='H')

Period('2012-01-01 19:00', 'H')

In [257]:
pd.Period('2012-1-1 19:00', freq='5H')

Period('2012-01-01 19:00', '5H')

Adding and subtracting integers from periods shifts the period by its own frequency. Arithmetic is not allowed between Period with different freq (span).

In [258]:
p = pd.Period('2012', freq='A-DEC')

In [259]:
p + 1

Period('2013', 'A-DEC')

In [260]:
p - 3

Period('2009', 'A-DEC')

In [261]:
p + 2

Period('2014', 'A-DEC')

In [262]:
p - 1

Period('2011', 'A-DEC')

    p == pd.Period('2012-01', freq='3M')
    
        ---------------------------------------------------------------------------
    IncompatibleFrequency                     Traceback (most recent call last)
    <ipython-input-271-ff54ce3238f5> in <module>()
    ----> 1 p == pd.Period('2012-01', freq='3M')

    pandas\src\period.pyx in pandas._period.Period.__richcmp__ (pandas\src\period.c:12534)()

    IncompatibleFrequency: Input has different freq=3M from Period(freq=A-DEC)


If Period freq is daily or higher (D, H, T, S, L, U, N), offsets and timedelta-like can be added if the result can have the same freq. Otherwise, ValueError will be raised.

In [263]:
p = pd.Period('2014-07-01 09:00', freq='H')

In [264]:
p + Hour(2)

Period('2014-07-01 11:00', 'H')

In [265]:
# timedelta(minutes=121) would raise an exception
p + timedelta(minutes=120) #datetime.timedelta 

Period('2014-07-01 11:00', 'H')

In [266]:
p + np.timedelta64(7200, 's')

Period('2014-07-01 11:00', 'H')

In [267]:
p + Minute(60)

Period('2014-07-01 10:00', 'H')

If Period has other freqs, only the same offsets can be added. Otherwise, ValueError will be raised.

In [268]:
p = pd.Period('2014-07', freq='M')

In [269]:
p

Period('2014-07', 'M')

In [270]:
p + MonthEnd(3)

Period('2014-10', 'M')

Taking the difference of Period instances with the same frequency will return the number of frequency units between them:

In [271]:
print pd.Period('2012', freq='A-DEC') - pd.Period('2002', freq='A-DEC')

10


### PeriodIndex and period_range

Regular sequences of Period objects can be collected in a PeriodIndex, which can be constructed using the period_range convenience function:

In [272]:
prng = pd.period_range('1/1/2011', '1/1/2012', freq='M')

In [273]:
prng

PeriodIndex(['2011-01', '2011-02', '2011-03', '2011-04', '2011-05', '2011-06',
             '2011-07', '2011-08', '2011-09', '2011-10', '2011-11', '2011-12',
             '2012-01'],
            dtype='int64', freq='M')

The PeriodIndex constructor can also be used directly:

In [274]:
pd.PeriodIndex(['2011-1', '2011-2', '2011-3'], freq='M')

PeriodIndex(['2011-01', '2011-02', '2011-03'], dtype='int64', freq='M')

Passing multiplied frequency outputs a sequence of Period which has multiplied span.

In [275]:
pd.PeriodIndex(start='2014-01', freq='3M', periods=4)

PeriodIndex(['2014-01', '2014-04', '2014-07', '2014-10'], dtype='int64', freq='3M')

Just like DatetimeIndex, a PeriodIndex can also be used to index pandas objects:

In [276]:
ps = pd.Series(np.random.randn(len(prng)), prng)

In [277]:
ps

2011-01   -1.012840
2011-02   -1.041833
2011-03   -0.527147
2011-04    0.204554
2011-05    0.278224
2011-06    1.900437
2011-07   -0.284101
2011-08   -1.354318
2011-09   -0.826196
2011-10    0.753280
2011-11   -0.594325
2011-12    0.411814
2012-01   -0.743065
Freq: M, dtype: float64

PeriodIndex supports addition and subtraction with the same rule as Period.

In [278]:
 idx = pd.period_range('2014-07-01 09:00', periods=5, freq='H')

In [279]:
idx

PeriodIndex(['2014-07-01 09:00', '2014-07-01 10:00', '2014-07-01 11:00',
             '2014-07-01 12:00', '2014-07-01 13:00'],
            dtype='int64', freq='H')

In [280]:
idx + Hour(2)

PeriodIndex(['2014-07-01 11:00', '2014-07-01 12:00', '2014-07-01 13:00',
             '2014-07-01 14:00', '2014-07-01 15:00'],
            dtype='int64', freq='H')

In [281]:
idx = pd.period_range('2014-07', periods=5, freq='M')

In [282]:
idx

PeriodIndex(['2014-07', '2014-08', '2014-09', '2014-10', '2014-11'], dtype='int64', freq='M')

In [283]:
idx + MonthEnd(3)

PeriodIndex(['2014-10', '2014-11', '2014-12', '2015-01', '2015-02'], dtype='int64', freq='M')

### PeriodIndex Partial String Indexing

You can pass in dates and strings to Series and DataFrame with PeriodIndex, in the same manner as DatetimeIndex. For details, refer to [DatetimeIndex Partial String Indexing](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-partialindexing).

In [284]:
ps

2011-01   -1.012840
2011-02   -1.041833
2011-03   -0.527147
2011-04    0.204554
2011-05    0.278224
2011-06    1.900437
2011-07   -0.284101
2011-08   -1.354318
2011-09   -0.826196
2011-10    0.753280
2011-11   -0.594325
2011-12    0.411814
2012-01   -0.743065
Freq: M, dtype: float64

In [285]:
ps['2011-01']

-1.012840058503071

In [286]:
ps[datetime(2011, 12, 25):]

2011-12    0.411814
2012-01   -0.743065
Freq: M, dtype: float64

In [287]:
ps['10/31/2011':'12/31/2011']

2011-10    0.753280
2011-11   -0.594325
2011-12    0.411814
Freq: M, dtype: float64

Passing a string representing a lower frequency than PeriodIndex returns partial sliced data.

In [288]:
ps['2011']

2011-01   -1.012840
2011-02   -1.041833
2011-03   -0.527147
2011-04    0.204554
2011-05    0.278224
2011-06    1.900437
2011-07   -0.284101
2011-08   -1.354318
2011-09   -0.826196
2011-10    0.753280
2011-11   -0.594325
2011-12    0.411814
Freq: M, dtype: float64

In [289]:
dfp = pd.DataFrame(np.random.randn(600,1),
                   columns=['A'],
                   index=pd.period_range('2013-01-01 9:00', periods=600, freq='T'))

In [290]:
dfp.head()

,A
2013-01-01 09:00,-0.047404
2013-01-01 09:01,0.059097
2013-01-01 09:02,-0.584466
2013-01-01 09:03,0.253171
2013-01-01 09:04,1.024422


In [291]:
dfp['2013-01-01 10H'].tail()

,A
2013-01-01 10:55,-0.376478
2013-01-01 10:56,-1.410736
2013-01-01 10:57,-1.060204
2013-01-01 10:58,-1.145796
2013-01-01 10:59,3.368170


As with DatetimeIndex, the endpoints will be included in the result. The example below slices data starting from 10:00 to 11:59.

In [292]:
dfp['2013-01-01 10H':'2013-01-01 11H'].tail()

,A
2013-01-01 11:55,-0.029076
2013-01-01 11:56,-0.580417
2013-01-01 11:57,0.476217
2013-01-01 11:58,-0.238291
2013-01-01 11:59,1.007003


### Frequency Conversion and Resampling with PeriodIndex

The frequency of Period and PeriodIndex can be converted via the asfreq method. Let’s start with the fiscal year 2011, ending in December:

In [293]:
 p = pd.Period('2011', freq='A-DEC')

In [294]:
p

Period('2011', 'A-DEC')

We can convert it to a monthly frequency. Using the how parameter, we can specify whether to return the starting or ending month:

In [295]:
p.asfreq('M', how='start')

Period('2011-01', 'M')

In [296]:
p.asfreq('M', how='end')

Period('2011-12', 'M')

The shorthands ‘s’ and ‘e’ are provided for convenience:

In [297]:
p.asfreq('M', 's')

Period('2011-01', 'M')

In [298]:
p.asfreq('M', 'e')

Period('2011-12', 'M')

Converting to a “super-period” (e.g., annual frequency is a super-period of quarterly frequency) automatically returns the super-period that includes the input period:

In [299]:
p = pd.Period('2011-12', freq='M')

In [300]:
p

Period('2011-12', 'M')

In [301]:
p.asfreq('A-NOV')

Period('2012', 'A-NOV')

Note that since we converted to an annual frequency that ends the year in November, the monthly period of December 2011 is actually in the 2012 A-NOV period.

Period conversions with anchored frequencies are particularly useful for working with various quarterly data common to economics, business, and other fields. Many organizations define quarters relative to the month in which their fiscal year starts and ends. Thus, first quarter of 2011 could start in 2010 or a few months into 2011. Via anchored frequencies, pandas works for all quarterly frequencies Q-JAN through Q-DEC.

Q-DEC define regular calendar quarters:

In [302]:
p = pd.Period('2012Q1', freq='Q-DEC')

In [303]:
p

Period('2012Q1', 'Q-DEC')

In [304]:
p.asfreq('D', 's')  #p.asfreq('D', how = 'start')

Period('2012-01-01', 'D')

In [305]:
p.asfreq('D', 'e') #p.asfreq('D', how = 'end')

Period('2012-03-31', 'D')

Q-MAR defines fiscal year end in March:

In [306]:
p = pd.Period('2011Q4', freq='Q-MAR')

In [307]:
p

Period('2011Q4', 'Q-MAR')

In [308]:
p.asfreq('D', 's')

Period('2011-01-01', 'D')

In [309]:
p.asfreq('D', 'e')

Period('2011-03-31', 'D')

## Converting between Representations

Timestamped data can be converted to PeriodIndex-ed data using to_period and vice-versa using to_timestamp:

In [310]:
rng = pd.date_range('1/1/2012', periods=5, freq='M')

In [311]:
rng

DatetimeIndex(['2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30',
               '2012-05-31'],
              dtype='datetime64[ns]', freq='M')

In [312]:
ts = pd.Series(np.random.randn(len(rng)), index=rng)

In [313]:
ts

2012-01-31   -1.552140
2012-02-29    0.521684
2012-03-31    0.558146
2012-04-30    1.908351
2012-05-31    1.254148
Freq: M, dtype: float64

In [314]:
ps = ts.to_period()

In [315]:
ps

2012-01   -1.552140
2012-02    0.521684
2012-03    0.558146
2012-04    1.908351
2012-05    1.254148
Freq: M, dtype: float64

In [316]:
ps.to_timestamp()

2012-01-01   -1.552140
2012-02-01    0.521684
2012-03-01    0.558146
2012-04-01    1.908351
2012-05-01    1.254148
Freq: MS, dtype: float64

Remember that ‘s’ and ‘e’ can be used to return the timestamps at the start or end of the period:

In [317]:
ps.to_timestamp('D', how='s')

2012-01-01   -1.552140
2012-02-01    0.521684
2012-03-01    0.558146
2012-04-01    1.908351
2012-05-01    1.254148
Freq: MS, dtype: float64

Converting between period and timestamp enables some convenient arithmetic functions to be used. In the following example, we convert a quarterly frequency with year ending in November to 9am of the end of the month following the quarter end

In [318]:
prng = pd.period_range('1990Q1', '2000Q4', freq='Q-NOV')

In [319]:
prng

PeriodIndex(['1990Q1', '1990Q2', '1990Q3', '1990Q4', '1991Q1', '1991Q2',
             '1991Q3', '1991Q4', '1992Q1', '1992Q2', '1992Q3', '1992Q4',
             '1993Q1', '1993Q2', '1993Q3', '1993Q4', '1994Q1', '1994Q2',
             '1994Q3', '1994Q4', '1995Q1', '1995Q2', '1995Q3', '1995Q4',
             '1996Q1', '1996Q2', '1996Q3', '1996Q4', '1997Q1', '1997Q2',
             '1997Q3', '1997Q4', '1998Q1', '1998Q2', '1998Q3', '1998Q4',
             '1999Q1', '1999Q2', '1999Q3', '1999Q4', '2000Q1', '2000Q2',
             '2000Q3', '2000Q4'],
            dtype='int64', freq='Q-NOV')

In [320]:
ts = pd.Series(np.random.randn(len(prng)), prng)

In [321]:
ts.head()

1990Q1    2.340595
1990Q2    0.029694
1990Q3   -1.491173
1990Q4   -0.151399
1991Q1   -0.236950
Freq: Q-NOV, dtype: float64

In [322]:
prng.asfreq('M', 'e') 

PeriodIndex(['1990-02', '1990-05', '1990-08', '1990-11', '1991-02', '1991-05',
             '1991-08', '1991-11', '1992-02', '1992-05', '1992-08', '1992-11',
             '1993-02', '1993-05', '1993-08', '1993-11', '1994-02', '1994-05',
             '1994-08', '1994-11', '1995-02', '1995-05', '1995-08', '1995-11',
             '1996-02', '1996-05', '1996-08', '1996-11', '1997-02', '1997-05',
             '1997-08', '1997-11', '1998-02', '1998-05', '1998-08', '1998-11',
             '1999-02', '1999-05', '1999-08', '1999-11', '2000-02', '2000-05',
             '2000-08', '2000-11'],
            dtype='int64', freq='M')

In [323]:
prng.asfreq('M', 'e') + 1

PeriodIndex(['1990-03', '1990-06', '1990-09', '1990-12', '1991-03', '1991-06',
             '1991-09', '1991-12', '1992-03', '1992-06', '1992-09', '1992-12',
             '1993-03', '1993-06', '1993-09', '1993-12', '1994-03', '1994-06',
             '1994-09', '1994-12', '1995-03', '1995-06', '1995-09', '1995-12',
             '1996-03', '1996-06', '1996-09', '1996-12', '1997-03', '1997-06',
             '1997-09', '1997-12', '1998-03', '1998-06', '1998-09', '1998-12',
             '1999-03', '1999-06', '1999-09', '1999-12', '2000-03', '2000-06',
             '2000-09', '2000-12'],
            dtype='int64', freq='M')

In [324]:
ts.index = (prng.asfreq('M', 'e') + 1).asfreq('H', 's') + 9

In [325]:
ts.head()

1990-03-01 09:00    2.340595
1990-06-01 09:00    0.029694
1990-09-01 09:00   -1.491173
1990-12-01 09:00   -0.151399
1991-03-01 09:00   -0.236950
Freq: H, dtype: float64

In [326]:
ts.to_timestamp().asfreq(BDay()).head(10)  #asfreq() will not create dates in between for PeriodIndex but with for Datetimeindex

1990-03-01 09:00:00    2.340595
1990-03-02 09:00:00         NaN
1990-03-05 09:00:00         NaN
1990-03-06 09:00:00         NaN
1990-03-07 09:00:00         NaN
1990-03-08 09:00:00         NaN
1990-03-09 09:00:00         NaN
1990-03-12 09:00:00         NaN
1990-03-13 09:00:00         NaN
1990-03-14 09:00:00         NaN
Freq: B, dtype: float64

## Representing out-of-bounds spans

If you have data that is outside of the Timestamp bounds, see [Timestamp limitations](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-timestamp-limits), then you can use a PeriodIndex and/or Series of Periods to do computations.

In [328]:
span = pd.period_range('1215-01-01', '1381-01-01', freq='D')

In [329]:
span

PeriodIndex(['1215-01-01', '1215-01-02', '1215-01-03', '1215-01-04',
             '1215-01-05', '1215-01-06', '1215-01-07', '1215-01-08',
             '1215-01-09', '1215-01-10',
             ...
             '1380-12-23', '1380-12-24', '1380-12-25', '1380-12-26',
             '1380-12-27', '1380-12-28', '1380-12-29', '1380-12-30',
             '1380-12-31', '1381-01-01'],
            dtype='int64', length=60632, freq='D')

To convert from a int64 based YYYYMMDD representation.

In [330]:
s = pd.Series([20121231, 20141130, 99991231])

In [331]:
def conv(x):
    return pd.Period(year = x // 10000, month = x//100 % 100, day = x%100, freq='D')

In [332]:
s.apply(conv)

0   2012-12-31
1   2014-11-30
2   9999-12-31
dtype: object

In [333]:
 s.apply(conv)[2]

Period('9999-12-31', 'D')

These can easily be converted to a PeriodIndex

In [334]:
span = pd.PeriodIndex(s.apply(conv))

In [335]:
span

PeriodIndex(['2012-12-31', '2014-11-30', '9999-12-31'], dtype='int64', freq='D')

## Time Zone Handling

Pandas provides rich support for working with timestamps in different time zones using pytz and dateutil libraries. dateutil support is new in 0.14.1 and currently only supported for fixed offset and tzfile zones. The default library is pytz. Support for dateutil is provided for compatibility with other applications e.g. if you use dateutil in other python packages.

### Working with Time Zones

By default, pandas objects are time zone unaware:

In [336]:
rng = pd.date_range('3/6/2012 00:00', periods=15, freq='D')

In [337]:
rng.tz is None

True

To supply the time zone, you can use the tz keyword to date_range and other functions. Dateutil time zone strings are distinguished from pytz time zones by starting with dateutil/.

* In pytz you can find a list of common (and less common) time zones using from pytz import common_timezones, all_timezones.
* dateutil uses the OS timezones so there isn’t a fixed list available. For common zones, the names are the same as pytz.


In [338]:
# pytz
rng_pytz = pd.date_range('3/6/2012 00:00', periods=10, freq='D',
                         tz='Europe/London')

In [339]:
rng_pytz.tz

<DstTzInfo 'Europe/London' LMT-1 day, 23:59:00 STD>

In [340]:
# dateutil
rng_dateutil = pd.date_range('3/6/2012 00:00', periods=10, freq='D',
                             tz='dateutil/Europe/London')

In [341]:
rng_dateutil.tz

tzfile('GB-Eire')

In [348]:
# dateutil - utc special case
import dateutil

rng_utc = pd.date_range('3/6/2012 00:00', periods=10, freq='D',
                        tz=dateutil.tz.tzutc())

In [349]:
rng_utc.tz

tzutc()

Note that the UTC timezone is a special case in dateutil and should be constructed explicitly as an instance of dateutil.tz.tzutc. You can also construct other timezones explicitly first, which gives you more control over which time zone is used:

In [354]:
# pytz
import pytz

tz_pytz = pytz.timezone('Europe/London')

In [355]:
rng_pytz = pd.date_range('3/6/2012 00:00', periods=10, freq='D',
             tz=tz_pytz)            

In [356]:
rng_pytz.tz == tz_pytz

True

In [357]:
# dateutil
tz_dateutil = dateutil.tz.gettz('Europe/London')

In [358]:
rng_dateutil = pd.date_range('3/6/2012 00:00', periods=10, freq='D',
                tz=tz_dateutil)

In [359]:
rng_dateutil.tz == tz_dateutil

True

Timestamps, like Python’s datetime.datetime object can be either time zone naive or time zone aware. Naive time series and DatetimeIndex objects can be localized using tz_localize:

In [360]:
ts = pd.Series(np.random.randn(len(rng)), rng)

In [361]:
ts_utc = ts.tz_localize('UTC')

In [362]:
ts_utc

2012-03-06 00:00:00+00:00    0.152267
2012-03-07 00:00:00+00:00   -0.836631
2012-03-08 00:00:00+00:00   -2.924732
2012-03-09 00:00:00+00:00   -0.804537
2012-03-10 00:00:00+00:00    1.072336
2012-03-11 00:00:00+00:00    0.438822
2012-03-12 00:00:00+00:00    0.496867
2012-03-13 00:00:00+00:00   -0.361741
2012-03-14 00:00:00+00:00   -1.296771
2012-03-15 00:00:00+00:00    0.445645
2012-03-16 00:00:00+00:00    0.449594
2012-03-17 00:00:00+00:00   -0.718936
2012-03-18 00:00:00+00:00    0.215125
2012-03-19 00:00:00+00:00   -0.006815
2012-03-20 00:00:00+00:00   -0.865533
Freq: D, dtype: float64

Again, you can explicitly construct the timezone object first. You can use the tz_convert method to convert pandas objects to convert tz-aware data to another time zone:

In [364]:
ts_utc.tz_convert('US/Eastern')

2012-03-05 19:00:00-05:00    0.152267
2012-03-06 19:00:00-05:00   -0.836631
2012-03-07 19:00:00-05:00   -2.924732
2012-03-08 19:00:00-05:00   -0.804537
2012-03-09 19:00:00-05:00    1.072336
2012-03-10 19:00:00-05:00    0.438822
2012-03-11 20:00:00-04:00    0.496867
2012-03-12 20:00:00-04:00   -0.361741
2012-03-13 20:00:00-04:00   -1.296771
2012-03-14 20:00:00-04:00    0.445645
2012-03-15 20:00:00-04:00    0.449594
2012-03-16 20:00:00-04:00   -0.718936
2012-03-17 20:00:00-04:00    0.215125
2012-03-18 20:00:00-04:00   -0.006815
2012-03-19 20:00:00-04:00   -0.865533
Freq: D, dtype: float64

**Warning:** Be wary of conversions between libraries. For some zones pytz and dateutil have different definitions of the zone. This is more of a problem for unusual timezones than for ‘standard’ zones like US/Eastern.

**Warning:** Be aware that a timezone definition across versions of timezone libraries may not be considered equal. This may cause problems when working with stored data that is localized using one version and operated on with a different version. See here for how to handle such a situation.

**Warning:** It is incorrect to pass a timezone directly into the datetime.datetime constructor (e.g., datetime.datetime(2011, 1, 1, tz=timezone('US/Eastern')). Instead, the datetime needs to be localized using the the localize method on the timezone.

Under the hood, all timestamps are stored in UTC. Scalar values from a DatetimeIndex with a time zone will have their fields (day, hour, minute) localized to the time zone. However, timestamps with the same UTC value are still considered to be equal even if they are in different time zones:

In [365]:
rng_eastern = rng_utc.tz_convert('US/Eastern')

In [366]:
rng_berlin = rng_utc.tz_convert('Europe/Berlin')

In [367]:
rng_eastern[5]

Timestamp('2012-03-10 19:00:00-0500', tz='US/Eastern', offset='D')

In [368]:
rng_berlin[5]

Timestamp('2012-03-11 01:00:00+0100', tz='Europe/Berlin', offset='D')

In [369]:
rng_eastern[5] == rng_berlin[5]

True

Like Series, DataFrame, and DatetimeIndex, Timestamp``s can be converted to other time zones using ``tz_convert:

In [370]:
rng_eastern[5]

Timestamp('2012-03-10 19:00:00-0500', tz='US/Eastern', offset='D')

In [371]:
rng_berlin[5]

Timestamp('2012-03-11 01:00:00+0100', tz='Europe/Berlin', offset='D')

In [372]:
rng_eastern[5].tz_convert('Europe/Berlin')

Timestamp('2012-03-11 01:00:00+0100', tz='Europe/Berlin')

Localization of Timestamp functions just like DatetimeIndex and Series:

In [373]:
rng[5]

Timestamp('2012-03-11 00:00:00', offset='D')

In [374]:
rng[5].tz_localize('Asia/Shanghai')

Timestamp('2012-03-11 00:00:00+0800', tz='Asia/Shanghai')

Operations between Series in different time zones will yield UTC Series, aligning the data on the UTC timestamps:

In [375]:
eastern = ts_utc.tz_convert('US/Eastern')

In [376]:
berlin = ts_utc.tz_convert('Europe/Berlin')

In [377]:
result = eastern + berlin

In [378]:
result.index

DatetimeIndex(['2012-03-06', '2012-03-07', '2012-03-08', '2012-03-09',
               '2012-03-10', '2012-03-11', '2012-03-12', '2012-03-13',
               '2012-03-14', '2012-03-15', '2012-03-16', '2012-03-17',
               '2012-03-18', '2012-03-19', '2012-03-20'],
              dtype='datetime64[ns, UTC]', freq='D')

To remove timezone from tz-aware DatetimeIndex, use tz_localize(None) or tz_convert(None). tz_localize(None) will remove timezone holding local time representations. tz_convert(None) will remove timezone after converting to UTC time.

In [379]:
didx = pd.DatetimeIndex(start='2014-08-01 09:00', freq='H', periods=10, tz='US/Eastern')

In [380]:
didx

DatetimeIndex(['2014-08-01 09:00:00-04:00', '2014-08-01 10:00:00-04:00',
               '2014-08-01 11:00:00-04:00', '2014-08-01 12:00:00-04:00',
               '2014-08-01 13:00:00-04:00', '2014-08-01 14:00:00-04:00',
               '2014-08-01 15:00:00-04:00', '2014-08-01 16:00:00-04:00',
               '2014-08-01 17:00:00-04:00', '2014-08-01 18:00:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', freq='H')

In [381]:
didx.tz_localize(None)

DatetimeIndex(['2014-08-01 09:00:00', '2014-08-01 10:00:00',
               '2014-08-01 11:00:00', '2014-08-01 12:00:00',
               '2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00', '2014-08-01 16:00:00',
               '2014-08-01 17:00:00', '2014-08-01 18:00:00'],
              dtype='datetime64[ns]', freq='H')

In [382]:
didx.tz_convert(None)

DatetimeIndex(['2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00', '2014-08-01 16:00:00',
               '2014-08-01 17:00:00', '2014-08-01 18:00:00',
               '2014-08-01 19:00:00', '2014-08-01 20:00:00',
               '2014-08-01 21:00:00', '2014-08-01 22:00:00'],
              dtype='datetime64[ns]', freq='H')

In [383]:
# tz_convert(None) is identical with tz_convert('UTC').tz_localize(None)
didx.tz_convert('UCT').tz_localize(None)

DatetimeIndex(['2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00', '2014-08-01 16:00:00',
               '2014-08-01 17:00:00', '2014-08-01 18:00:00',
               '2014-08-01 19:00:00', '2014-08-01 20:00:00',
               '2014-08-01 21:00:00', '2014-08-01 22:00:00'],
              dtype='datetime64[ns]', freq='H')

### Ambiguous Times when Localizing

In some cases, localize cannot determine the DST and non-DST hours when there are duplicates. This often happens when reading files or database records that simply duplicate the hours. Passing ambiguous='infer' (infer_dst argument in prior releases) into tz_localize will attempt to determine the right offset. Below the top example will fail as it contains ambiguous times and the bottom will infer the right offset.

In [384]:
rng_hourly = pd.DatetimeIndex(['11/06/2011 00:00', '11/06/2011 01:00',
                                '11/06/2011 01:00', '11/06/2011 02:00',
                                '11/06/2011 03:00'])

This will fail as there are ambiguous times

In [387]:
#rng_hourly.tz_localize('US/Eastern')

#AmbiguousTimeError: Cannot infer dst time from Timestamp('2011-11-06 01:00:00'), try using the 'ambiguous' argument

In [388]:
rng_hourly_eastern = rng_hourly.tz_localize('US/Eastern', ambiguous='infer')

In [389]:
rng_hourly_eastern.tolist()

[Timestamp('2011-11-06 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2011-11-06 01:00:00-0400', tz='US/Eastern'),
 Timestamp('2011-11-06 01:00:00-0500', tz='US/Eastern'),
 Timestamp('2011-11-06 02:00:00-0500', tz='US/Eastern'),
 Timestamp('2011-11-06 03:00:00-0500', tz='US/Eastern')]

In addition to ‘infer’, there are several other arguments supported. Passing an array-like of bools or 0s/1s where True represents a DST hour and False a non-DST hour, allows for distinguishing more than one DST transition (e.g., if you have multiple records in a database each with their own DST transition). Or passing ‘NaT’ will fill in transition times with not-a-time values. These methods are available in the DatetimeIndex constructor as well as tz_localize.

In [390]:
rng_hourly_dst = np.array([1, 1, 0, 0, 0])

In [391]:
rng_hourly.tz_localize('US/Eastern', ambiguous=rng_hourly_dst).tolist()

[Timestamp('2011-11-06 00:00:00-0400', tz='US/Eastern'),
 Timestamp('2011-11-06 01:00:00-0400', tz='US/Eastern'),
 Timestamp('2011-11-06 01:00:00-0500', tz='US/Eastern'),
 Timestamp('2011-11-06 02:00:00-0500', tz='US/Eastern'),
 Timestamp('2011-11-06 03:00:00-0500', tz='US/Eastern')]

In [392]:
rng_hourly.tz_localize('US/Eastern', ambiguous='NaT').tolist()

[Timestamp('2011-11-06 00:00:00-0400', tz='US/Eastern'),
 NaT,
 NaT,
 Timestamp('2011-11-06 02:00:00-0500', tz='US/Eastern'),
 Timestamp('2011-11-06 03:00:00-0500', tz='US/Eastern')]

In [393]:
didx = pd.DatetimeIndex(start='2014-08-01 09:00', freq='H', periods=10, tz='US/Eastern')

In [394]:
didx

DatetimeIndex(['2014-08-01 09:00:00-04:00', '2014-08-01 10:00:00-04:00',
               '2014-08-01 11:00:00-04:00', '2014-08-01 12:00:00-04:00',
               '2014-08-01 13:00:00-04:00', '2014-08-01 14:00:00-04:00',
               '2014-08-01 15:00:00-04:00', '2014-08-01 16:00:00-04:00',
               '2014-08-01 17:00:00-04:00', '2014-08-01 18:00:00-04:00'],
              dtype='datetime64[ns, US/Eastern]', freq='H')

In [395]:
didx.tz_localize(None)

DatetimeIndex(['2014-08-01 09:00:00', '2014-08-01 10:00:00',
               '2014-08-01 11:00:00', '2014-08-01 12:00:00',
               '2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00', '2014-08-01 16:00:00',
               '2014-08-01 17:00:00', '2014-08-01 18:00:00'],
              dtype='datetime64[ns]', freq='H')

In [396]:
didx.tz_convert(None)

DatetimeIndex(['2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00', '2014-08-01 16:00:00',
               '2014-08-01 17:00:00', '2014-08-01 18:00:00',
               '2014-08-01 19:00:00', '2014-08-01 20:00:00',
               '2014-08-01 21:00:00', '2014-08-01 22:00:00'],
              dtype='datetime64[ns]', freq='H')

In [397]:
# tz_convert(None) is identical with tz_convert('UTC').tz_localize(None)
didx.tz_convert('UCT').tz_localize(None)

DatetimeIndex(['2014-08-01 13:00:00', '2014-08-01 14:00:00',
               '2014-08-01 15:00:00', '2014-08-01 16:00:00',
               '2014-08-01 17:00:00', '2014-08-01 18:00:00',
               '2014-08-01 19:00:00', '2014-08-01 20:00:00',
               '2014-08-01 21:00:00', '2014-08-01 22:00:00'],
              dtype='datetime64[ns]', freq='H')

### TZ aware Dtypes

New in version 0.17.0.

Series/DatetimeIndex with a timezone naive value are represented with a dtype of datetime64[ns].

In [398]:
s_naive = pd.Series(pd.date_range('20130101',periods=3))

In [399]:
s_naive

0   2013-01-01
1   2013-01-02
2   2013-01-03
dtype: datetime64[ns]

Series/DatetimeIndex with a timezone aware value are represented with a dtype of datetime64[ns, tz].

In [400]:
s_aware = pd.Series(pd.date_range('20130101',periods=3,tz='US/Eastern'))

In [401]:
s_aware

0   2013-01-01 00:00:00-05:00
1   2013-01-02 00:00:00-05:00
2   2013-01-03 00:00:00-05:00
dtype: datetime64[ns, US/Eastern]

Both of these Series can be manipulated via the .dt accessor, see here.

For example, to localize and convert a naive stamp to timezone aware.

In [402]:
s_naive.dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

0   2012-12-31 19:00:00-05:00
1   2013-01-01 19:00:00-05:00
2   2013-01-02 19:00:00-05:00
dtype: datetime64[ns, US/Eastern]

Further more you can .astype(...) timezone aware (and naive). This operation is effectively a localize AND convert on a naive stamp, and a convert on an aware stamp.

In [403]:
# localize and convert a naive timezone
s_naive.astype('datetime64[ns, US/Eastern]')

0   2012-12-31 19:00:00-05:00
1   2013-01-01 19:00:00-05:00
2   2013-01-02 19:00:00-05:00
dtype: datetime64[ns, US/Eastern]

In [404]:
# make an aware tz naive
s_aware.astype('datetime64[ns]')

0   2013-01-01 05:00:00
1   2013-01-02 05:00:00
2   2013-01-03 05:00:00
dtype: datetime64[ns]

In [405]:
# convert to a new timezone
s_aware.astype('datetime64[ns, CET]')

0   2013-01-01 06:00:00+01:00
1   2013-01-02 06:00:00+01:00
2   2013-01-03 06:00:00+01:00
dtype: datetime64[ns, CET]

**Note:** Using the .values accessor on a Series, returns an numpy array of the data. These values are converted to UTC, as numpy does not currently support timezones (even though it is printing in the local timezone!).

In [406]:
s_naive.values

array(['2013-01-01T00:00:00.000000000', '2013-01-02T00:00:00.000000000',
       '2013-01-03T00:00:00.000000000'], dtype='datetime64[ns]')

In [407]:
s_aware.values

array(['2013-01-01T05:00:00.000000000', '2013-01-02T05:00:00.000000000',
       '2013-01-03T05:00:00.000000000'], dtype='datetime64[ns]')

Further note that once converted to a numpy array these would lose the tz tenor.

In [408]:
pd.Series(s_aware.values)

0   2013-01-01 05:00:00
1   2013-01-02 05:00:00
2   2013-01-03 05:00:00
dtype: datetime64[ns]

However, these can be easily converted

In [409]:
pd.Series(s_aware.values).dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

0   2013-01-01 00:00:00-05:00
1   2013-01-02 00:00:00-05:00
2   2013-01-03 00:00:00-05:00
dtype: datetime64[ns, US/Eastern]